### Load libraries and initialize constants...

In [1]:
import matplotlib.pyplot as plt, matplotlib.colors as colors, matplotlib.path as mpath
from matplotlib.patches import FancyArrowPatch, Circle, Wedge, Polygon, PathPatch
from numpy.polynomial.polynomial import polyfit
from scipy.optimize import curve_fit, fmin_cobyla

import math, numbers, string, re, itertools, random, copy
import numpy as np, pandas as pd
import scipy.stats as stats
import collections as collect
import seaborn as sns
import networkx as nx
import colorcet as cc
import functools as func
import operator as operate
from pprint import pprint

%matplotlib inline
sns.set_style("dark")

GRAPH_WIDTH = 20
GRAPH_HEIGHT = 20
GRAPH_DOTS_PER_INCH = 160
MAX_STEP_COUNT = 5
EDGE_CURVE_OFFSET_RATIO = 0.1

SPLIT_CIRCLE_ANGLE_0 = 30
SPLIT_CIRCLE_ANGLE_1 = 150
SPLIT_CIRCLE_ANGLE_2 = 270

CGRAPH_RADIUS_MULTIPLIER = 1.2
CGRAPH_LABEL_RADIUS_MULTIPLIER = 1.25
CGRAPH_LEGEND_RADIUS_MULTIPLIER = 1.35

GRAPH_TITLE = 'Entropy Map:'
GRAPH_DATASET_LABEL = 'Entropy rate for\ntotal dataset:\n%.2f bits'
GRAPH_TITLE_2 = "Mutual Information Map\nfor '%s':"
GRAPH_DATASET_LABEL_2 = "Entropy rate for\n%s field:\n%.2f bits"
RADIUS_FOR_TOTAL_ENTROPY = 0.07
RADIUS_FOR_TOTAL_MUTUAL_INFORMATION = 0.07

BACKGROUND_COLOR = '#000000' #'#002000'#'#222222' #'#111111' 
FONT_COLOR = 'white'
ENTROPY_BACKGROUND_COLOR = 'lightgray'

ENTROPY_CMAP = cc.cm['bkr'] # 'RdYlBu' # 'coolwarm' # 'YlOrRd' # YlOrRd' #'Set3' # 'binary
ENTROPY_CMAP2 = cc.cm['bky'] # 'diverging_linear_bjy_30_90_c45' # 'bgy'  # 'bjy' # 'RdYlGn'
TAB_CMAP = plt.get_cmap('tab20')

LETTER_COLOR_DICT = { c: TAB_CMAP(i/20.0) for (i, c) in enumerate(string.ascii_uppercase[:20]) }
HIGHLIGHT_DICT = {
        'out2n':'#550000', 'out1n':'#550000', 'out1p':'#550000', 'out2p':'#550000'
    }

### Load in data from file...

In [ ]:
#aDF = pd.read_csv("Edmonton_Arts_Council_10_Year_Strategic_Plan_-_Edmonton_Insight_Community.csv")
#cL = list(aDF.columns)
#for i, c in enumerate(cL):
#    c = re.sub('^Q\d+\w?(_ArtsCouncil)?_', '', c)
#    c = re.sub('\s*\(Source\:\s.*\)$', '', c)
#    cL[i] = c
#aDF.columns = pd.Index(cL)
#aDF = aDF[list(aDF.columns[2:10])]

#aDF = pd.read_csv('nyc_subway_weather.csv')
aDF = pd.read_csv('winequality-red.csv', sep=';')
#aDF = aDF[aDF.columns[:14]]
#aDF = aDF[['UNIT', 'DATEn', 'TIMEn', 'ENTRIESn_hourly', 'EXITSn_hourly', 'weekday', 
#          'conds', 'fog', 'precipi', 'pressurei', 'tempi', 'wspdi']]

In [ ]:
aDF.head()

In [ ]:
aDF.describe(include='all')

### Alternatively, produce a synthetic data set for testing

In [ ]:
TEST_CATEGORIES = ['cA', 'cB', 'cC', 'cD', 'cE', 'cF', 'cG', 'cH']

def get_random_cat_p_ranges():
    nL = [0.2 + random.random() * 0.8 for i in range(8)]
    pL, d = [0], sum(nL)
    for n in nL: pL.append(pL[-1]+n/d) 
    return pL

def get_cat_by_defcat_ranges(category_ranges):
    c, v = None, random.random() 
    for i in range(8):
        if category_ranges[i] <= v < category_ranges[i+1]:
            c = TEST_CATEGORIES[i]
            break
    return c

def get_modelDict(r0_count=2, rp1_count=2, rp2_count=1, rn2_count=1):
    fi, fDcprL, rDpL = 1, {}, {}
    for c0 in range(r0_count):
        fDcprL['f%d' % (fi)] = get_random_cat_p_ranges()
        fi += 1
    for c1 in range(rp1_count):
        fDcprL['f%da' % (fi)] = get_random_cat_p_ranges()
        fDcprL['f%db' % (fi)] = get_random_cat_p_ranges()
        rDpL[('f%da' % (fi), 'f%db' % (fi))] = [random.random() for i in range(8)] 
        fi += 1
    for c2 in range(rp2_count):
        fDcprL['f%da' % (fi)] = get_random_cat_p_ranges()
        fDcprL['f%db' % (fi)] = get_random_cat_p_ranges()
        fDcprL['f%dc' % (fi)] = get_random_cat_p_ranges()
        rDpL[('f%da' % (fi), 'f%db' % (fi))] = [random.random() for i in range(8)] 
        rDpL[('f%db' % (fi), 'f%dc' % (fi))] = [random.random() for i in range(8)] 
        fi += 1
    for c3 in range(rn2_count):
        fDcprL['f%da' % (fi)] = get_random_cat_p_ranges()
        fDcprL['f%db' % (fi)] = get_random_cat_p_ranges()
        fDcprL['f%dc' % (fi)] = get_random_cat_p_ranges()
        rDpL[('f%da' % (fi), 'f%db' % (fi), 'f%dc' % (fi))] = [random.random() for i in range(8)]
        fi += 1

    return {'fields': fDcprL, 'relations': rDpL}
            
def get_sampleDFrame(modelD, sample_count):
    fDcprL, rDpL, fn1, fn2, fDvL = modelD['fields'], modelD['relations'], None, None, {}
    for f in sorted(fDcprL):
        rn2, rn1, fDvL[f] = (fn2, fn1, f), (fn1, f), []
        for i in range(sample_count):
            v, rv = None, random.random()
            if rn2 in rDpL:
                vn2, vn1 = fDvL[fn2][i], fDvL[fn1][i]    
                v0 = TEST_CATEGORIES[(TEST_CATEGORIES.index(vn1)+TEST_CATEGORIES.index(vn2)+1) % 8]
                tv = rDpL[rn2][TEST_CATEGORIES.index(vn2)]
                v = v0 if rv < tv else get_cat_by_defcat_ranges(fDcprL[f])
            elif rn1 in rDpL:
                vn1 = fDvL[fn1][i]
                v0 = TEST_CATEGORIES[(TEST_CATEGORIES.index(vn1)+1) % 8]
                tv = rDpL[rn1][TEST_CATEGORIES.index(vn1)]
                v = v0 if rv < tv else get_cat_by_defcat_ranges(fDcprL[f])
            else: 
                v = get_cat_by_defcat_ranges(fDcprL[f])
            fDvL[f].append(v)
        fn2 = fn1
        fn1 = f  
    return pd.DataFrame(fDvL)
    
mD = get_modelDict(0, 0, 2, 2)

mD2 = copy.deepcopy(mD)

#mD2['fields']['f1a'] = get_random_cat_p_ranges()
#mD2['fields']['f1b'] = get_random_cat_p_ranges()
#mD2['fields']['f1c'] = get_random_cat_p_ranges()
#mD2['fields']['f2a'] = get_random_cat_p_ranges()
#mD2['fields']['f2b'] = get_random_cat_p_ranges()
#mD2['fields']['f2c'] = get_random_cat_p_ranges()
#mD2['fields']['f3a'] = get_random_cat_p_ranges()
#mD2['fields']['f3b'] = get_random_cat_p_ranges()
#mD2['fields']['f3c'] = get_random_cat_p_ranges()
#mD2['fields']['f4a'] = get_random_cat_p_ranges()
#mD2['fields']['f4b'] = get_random_cat_p_ranges()
#mD2['fields']['f4c'] = get_random_cat_p_ranges()
#mD2['relations'][('f2a', 'f2b')] = [random.random() for i in range(8)]
mD2['relations'][('f2b', 'f2c')] = [random.random() for i in range(8)]
mD2['relations'][('f4a', 'f4b', 'f4c')] = [random.random() for i in range(8)]

eDF = get_sampleDFrame(mD, 10000)
eDFc = get_sampleDFrame(mD2, 10000)
eDF.describe(include='all')

In [ ]:
eDFc.describe(include='all')

### Discretize data...

In [ ]:
def discretize_data_SS(SS):
    SS2, dSS, u = SS, SS.describe(include='all'), None
    if 'unique' in dSS.index:
        SS = SS.fillna('Not Recorded')
        vDnv = {k: 'v'+str(i+1) for i, k in enumerate(sorted(collect.Counter(SS).keys()))}
        SS2 = SS.apply(lambda v: vDnv[v])
    else:
        is_not_assigned = True
        q125, q375, q625, q875 = SS.quantile(0.125), SS.quantile(0.375), SS.quantile(0.625), SS.quantile(0.875)
        q250, q500, q750 = dSS['25%'], dSS['50%'], dSS['75%']
        if (q125 != q250 != q375 != q500 != q625 != q750 != q875):
            SS2 = SS.apply(lambda v: 'e1' if v<q125 else ('e2' if v<q250 else ('e3' if v<q375 else ('e4' if v<q500 \
                           else ('e5' if v<q625 else ('e6' if v<q750 else ('e7' if v<q875 else 'e8')))))))
            is_not_assigned = False
        if is_not_assigned and (q250 != q500 != q750):
            SS2 = SS.apply(lambda v: 'q1' if v<q250 else ('q2' if v<q500 else ('q3' if v<q750 else 'q4')))
            is_not_assigned = False
        q330, q660 = SS.quantile(0.33), SS.quantile(0.66)
        if is_not_assigned and (q330 != q660):
            SS2 = SS.apply(lambda v: 't1' if v < q330 else ('t2' if v < q660 else 't3'))
            is_not_assigned = False
        q000, q1000 = SS.min(), SS.max()    
        if is_not_assigned and q000 != q500 != q1000:
            SS2 = SS.apply(lambda v: 'h1' if v < q500 else 'h2')
            is_not_assigned = False
        if is_not_assigned:
            cval = sorted(collect.Counter(SS).items(), key=lambda kv: kv[1])[-1][0]
            SS2 = SS.apply(lambda v: 'mc' if v == cval else 'lc')
            is_not_assigned = False
        if is_not_assigned:
            SS2 = SS.apply(lambda v: 'sv')
    return SS2

eDF = aDF.apply(discretize_data_SS)
eDF.describe(include='all')

### Subset dataframe for purposes of comparing distributions

In [ ]:
def subset_DF_on_field_and_value(DF, field, values):
    ssaDF = DF.loc[DF[field].isin(values)].drop(field, axis=1)
    ssbDF = DF.loc[~DF[field].isin(values)].drop(field, axis=1)
    mDF = DF.drop(field, axis=1)
    return ssaDF, ssbDF, mDF

#eDF, eDFc, _ = subset_DF_on_field_and_value(eDF, 'precipi', ['lc'])
#eDF, eDFc, _ = subset_DF_on_field_and_value(eDF, 'weekday', ['lc'])
eDF, eDFc, _ = subset_DF_on_field_and_value(eDF, 'quality', ['t3'])
#eDF, eDFc, _ = subset_DF_on_field_and_value(eDF, 'pH', ['e7','e8'])
#eDF, eDFc, _ = subset_DF_on_field_and_value(eDF, 'density', ['e1','e2'])
#eDF, eDFc, _ = subset_DF_on_field_and_value(eDF, 'sulphates', #'total sulfur dioxide', 
#        ['e1','e2','e3','e4'])

eDF.describe(include='all')

In [ ]:
eDFc.describe(include='all')

### Entropy calculation utility functions...

In [ ]:
def get_unique_values_for_SS(SS):
    return pd.Series(sorted([v for v in set(SS)]))

def get_unique_values_for_DF(DF):
    return pd.Series([get_unique_values_for_SS(DF[c]) for c in DF.columns], index=DF.columns)

def superkey_DF(DF):
    if type(DF) != pd.DataFrame: raise TypeError('Superkey argument must be a DataFrame')
    return pd.Series(list(zip(*[DF[c] for c in DF.keys()])), name=tuple(DF.keys()))

def get_E_for_SS(SS, cSS=None, tflag=None):
    pC = collect.Counter(SS)
    e = None    
    if cSS is None:
        if len(pC) <= 1: raise AssertionError('Series must have at least 2 states')
        e = stats.entropy([v + 0.0000001 for v in pC.values()], base=2)
    elif cSS is not None:
        qC = collect.Counter(cSS)
        kL = list(set(pC.keys()) | set(qC.keys()))
        if len(kL) <= 1: raise AssertionError('Series must have at least 2 states')
        pk, qk = [], []
        for k in kL:                
            pk.append(((pC[k] if k in pC else 0) + 0.0000001))
            qk.append(((qC[k] if k in qC else 0) + 0.0000001))
        e = stats.entropy(pk, base=2) + stats.entropy(pk, qk=qk, base=2)
    return e

def get_max_E_for_SS(SS, cSS=None, tflag=None):
    pC = collect.Counter(SS)
    e, ps, pl = None, len(SS), len(pC)    
    if cSS is None:
        if len(pC) <= 1: raise AssertionError('Series must have at least 2 states')
        e = stats.entropy([ps/pl + 0.0000001]*pl, base=2)
    elif cSS is not None:
        qC = collect.Counter(cSS)
        kL, qs, ql = list(set(pC.keys()) | set(qC.keys())), len(cSS), len(qC)
        if len(kL) <= 1: raise AssertionError('Series must have at least 2 states')
        pk, qk = [], []
        for k in kL:                
            pk.append(((ps/pl if k in pC else 0) + 0.0000001))
            qk.append(((qs/ql if k in qC else 0) + 0.0000001))
        e = stats.entropy(pk, base=2) + stats.entropy(pk, qk=qk, base=2)
    return e

In [ ]:
#print(get_E_for_SS(eDF['citric acid']))
#print(get_max_E_for_SS(eDF['citric acid']))

In [ ]:
def get_jE_for_DF(DF, cDF=None, tflag=None):
    SS, cSS = superkey_DF(DF), superkey_DF(cDF) if cDF is not None else None 
    return get_E_for_SS(SS, cSS=cSS, tflag=tflag)

def get_Es_for_DF(DF, cDF=None, tflag=None):
    cL, eL = DF.columns, None
    if cDF is None: eL = [get_E_for_SS(DF[c], tflag=tflag) for c in cL]
    else: eL = [get_E_for_SS(DF[c], cSS=cDF[c], tflag=tflag) for c in cL]
    return pd.Series(eL, index=cL)

def get_max_Es_for_DF(DF, cDF=None, tflag=None):
    cL, eL = DF.columns, None
    if cDF is None: eL = [get_max_E_for_SS(DF[c], tflag=tflag) for c in cL]
    else: eL = [get_max_E_for_SS(DF[c], cSS=cDF[c], tflag=tflag) for c in cL]
    return pd.Series(eL, index=cL)

def get_2v_jEs_for_DF(DF, cDF=None, tflag=None):
    cDje = {}
    for i, c in enumerate(DF.columns):
        cDje[c] = []
        for j, c2 in enumerate(DF.columns):
            if cDF is None:
                cDje[c].append(get_jE_for_DF(DF[[c, c2]], tflag=tflag) if i > j else np.NaN)
            else:
                cDje[c].append(get_jE_for_DF(DF[[c, c2]], cDF=cDF[[c, c2]], tflag=tflag) 
                               if i > j else np.NaN)
    return pd.DataFrame(cDje, columns=DF.columns, index=DF.columns)

def get_2v_MIs_for_DF(DF, cDF=None, tflag=None):
    eSS = get_Es_for_DF(DF, cDF=cDF, tflag=tflag)
    jeDF = get_2v_jEs_for_DF(DF, cDF=cDF, tflag=tflag)    
    return pd.DataFrame({c: [eSS[c]+eSS[c2]-jeDF[c][c2] for c2 in DF.columns] for c in DF.columns}, 
                        index = DF.columns)

def get_max_2v_MIs_for_DF(DF, cDF=None, tflag=None):
    eSS = get_max_Es_for_DF(DF, cDF=cDF, tflag=tflag)
    return pd.DataFrame({DF.columns[i]: [min(eSS.iloc[j], eSS.iloc[i]) if i > j else np.NaN
                             for j in range(DF.shape[1])] for i in range(DF.shape[1])}, 
                        index = DF.columns)

def get_3v_jEs_for_DF(DF, cDF=None, tflag=None):
    c1c2c3L, je3L = [], []
    for i in range(DF.shape[1]):
        for j in range(i+1, DF.shape[1]):
            for k in range(j+1, DF.shape[1]):
                c1, c2, c3, je3 = DF.columns[i], DF.columns[j], DF.columns[k], None
                if cDF is None:
                    je3 = get_jE_for_DF( DF[[c1,c2,c3]], tflag=tflag )
                else:
                    je3 = get_jE_for_DF(DF[[c1,c2,c3]], cDF=cDF[[c1,c2,c3]], tflag=tflag)
                c1c2c3L.append((c1,c2,c3))
                je3L.append(je3)
    return pd.Series(je3L, index=c1c2c3L)

def get_3v_MIs_for_DF(DF, cDF=None, tflag=None):
    eSS = get_Es_for_DF(DF, cDF=cDF, tflag=tflag)
    jeDF = get_2v_jEs_for_DF(DF, cDF=cDF, tflag=tflag)
    tjeSS = get_3v_jEs_for_DF(DF, cDF=cDF, tflag=tflag)    
    tmiL, cxcyczL = [], []
    for i in range(DF.shape[1]):
        for j in range(i+1, DF.shape[1]):
            for k in range(j+1, DF.shape[1]):
                cX, cY ,cZ = DF.columns[i], DF.columns[j], DF.columns[k]
                eX, eY, eZ = eSS[cX], eSS[cY], eSS[cZ]
                jeXY, jeXZ, jeYZ = jeDF[cY][cX], jeDF[cZ][cX], jeDF[cZ][cY]  
                tje = tjeSS[(cX,cY,cZ)]
                tmi = tje+eX+eY+eZ-jeXZ-jeXY-jeYZ
                tmiL.append(tmi)
                cxcyczL.append((cX,cY,cZ))
    return pd.Series(tmiL, index=cxcyczL)

def get_max_3v_MIs_for_DF(DF, cDF=None, tflag=None):
    eSS, tmiL, cxcyczL = get_max_Es_for_DF(DF, cDF=cDF, tflag=tflag), [], []
    for i in range(DF.shape[1]):
        for j in range(i+1, DF.shape[1]):
            for k in range(j+1, DF.shape[1]):
                cX, cY ,cZ = DF.columns[i], DF.columns[j], DF.columns[k]
                tmiL.append(min(eSS[cX], eSS[cY], eSS[cZ]))
                cxcyczL.append((cX,cY,cZ))
    return pd.Series(tmiL, index=cxcyczL)

In [ ]:
#get_max_3v_MIs_for_DF(eDF, eDFc)

In [ ]:
#get_3v_MIs_for_DF(eDF, eDFc)

In [ ]:
def get_2v_jEs_for_DF_and_target_field(DF, target_field, cDF=None):
    jeDF = get_2v_jEs_for_DF(DF, cDF=cDF).fillna(0)
    return (jeDF+jeDF.T)[target_field].drop(target_field)

def get_2v_MIs_for_DF_and_target_field(DF, target_field, cDF=None):
    eSS = get_Es_for_DF(DF, cDF=cDF) 
    jeDF =  get_2v_jEs_for_DF_and_target_field(DF, target_field, cDF=cDF)
    return pd.Series([eSS[target_field]+eSS[c]-jeDF[c] for c in jeDF.index], index=jeDF.index)

def get_3v_jEs_for_DF_and_target_field(DF, target_field, cDF=None):
    cL, cDtje = [c for c in DF.columns if c != target_field], {}
    for i in range(len(cL)):
        cDtje[cL[i]] = [np.NaN]*len(cL)
        for j in range(i):
            fL = [target_field, cL[i], cL[j]]
            if cDF is None:
                cDtje[cL[i]][j] = get_jE_for_DF( DF[fL] )
            else:
                cDtje[cL[i]][j] = get_jE_for_DF( DF[fL], cDF[fL] )                
    return pd.DataFrame(cDtje, columns=cL, index=cL)

def get_3v_MIs_for_DF_and_target_field(DF, target_field, cDF=None):
    eSS = get_Es_for_DF(DF, cDF=cDF)
    jeDF = get_2v_jEs_for_DF(DF, cDF=cDF)
    tjeDF = get_3v_jEs_for_DF_and_target_field(DF, target_field, cDF=cDF)
    tmiDF = pd.DataFrame({c: [np.NaN]*tjeDF.shape[0] for c in tjeDF.columns}, columns=tjeDF.columns, index=tjeDF.index)
    t2 = DF.columns.get_loc(target_field)
    for i in range(tjeDF.shape[1]):
        cX, i2 = tjeDF.columns[i], DF.columns.get_loc(tjeDF.columns[i])
        for j in range(i+1, tjeDF.shape[1]):
            cY, j2 = tjeDF.columns[j], DF.columns.get_loc(tjeDF.columns[j])
            eT, eX, eY = eSS[target_field], eSS[cX], eSS[cY]
            jeXY, jeTX, jeTY = jeDF[cY][cX], None, None
            if t2 < i2 < j2: jeTX, jeTY = jeDF[cX][target_field], jeDF[cY][target_field]
            elif i2 < t2 < j2: jeTX, jeTY = jeDF[target_field][cX], jeDF[cY][target_field]  
            elif i2 < j2 < t2: jeTX, jeTY = jeDF[target_field][cX], jeDF[target_field][cY]
            tmiDF[cY][cX] = tjeDF[cY][cX]+eT+eX+eY-jeTX-jeTY-jeXY
    return tmiDF

def get_3v_2WC_info_for_DF_and_target_field(DF, target_field, cDF=None):
    jeDF, eT = get_2v_jEs_for_DF(DF, cDF=cDF), None
    tjeDF = get_3v_jEs_for_DF_and_target_field(DF, target_field, cDF=cDF)
    twcDF = pd.DataFrame({c: [np.NaN]*tjeDF.shape[0] for c in tjeDF.columns}, columns=tjeDF.columns, index=tjeDF.index)
    if cDF is None:
        eT = get_E_for_SS(DF[target_field])
    else:
        eT = get_E_for_SS(DF[target_field], cDF[target_field])
    for i in range(tjeDF.shape[1]):
        for j in range(i+1, tjeDF.shape[1]):
            jeUV = jeDF[tjeDF.columns[j]][tjeDF.columns[i]] 
            jeUVT =tjeDF[tjeDF.columns[j]][tjeDF.columns[i]]
            twcDF[tjeDF.columns[j]][tjeDF.columns[i]] = jeUV+eT-jeUVT
    return twcDF

def get_4v_MIs_for_DF_and_target_field(DF, target_field, cDF=None):
    eSS = get_Es_for_DF(DF, cDF=cDF)
    jeDF = get_2v_jEs_for_DF(DF, cDF=cDF)
    tjeDF = get_3v_jEs_for_DF_and_target_field(DF, target_field, cDF=cDF)
    qmiL, cxcucvL = [], []
    t2 = DF.columns.get_loc(target_field)
    for i in range(tjeDF.shape[1]):
        cX, i2 = tjeDF.columns[i], DF.columns.get_loc(tjeDF.columns[i])
        for j in range(i+1, tjeDF.shape[1]):
            cU, j2 = tjeDF.columns[j], DF.columns.get_loc(tjeDF.columns[j])
            for k in range(j+1, tjeDF.shape[1]):
                cV, k2 = tjeDF.columns[k], DF.columns.get_loc(tjeDF.columns[k])
                eX, eU, eV, eT = eSS[cX], eSS[cU], eSS[cV], eSS[target_field]
                jeXU, jeXV, jeUV = jeDF[cU][cX], jeDF[cV][cX], jeDF[cV][cU]
                jeXUT, jeXVT, jeUVT = tjeDF[cU][cX], tjeDF[cV][cX], tjeDF[cV][cU]
                cL1, cL2 = [cX, cU, cV], [cX, cU, cV, target_field]
                if cDF is None:
                    jeXUV = get_jE_for_DF(DF[cL1])
                    jeXUVT = get_jE_for_DF(DF[cL2])
                else:
                    jeXUV = get_jE_for_DF(DF[cL1], cDF=cDF[cL1])
                    jeXUVT = get_jE_for_DF(DF[cL2], cDF=cDF[cL2])
                if t2 < i2 < j2 < k2: jeXT, jeUT, jeVT = jeDF[cX][target_field], jeDF[cU][target_field], jeDF[cV][target_field]
                elif i2 < t2 < j2 < k2: jeXT, jeUT, jeVT = jeDF[target_field][cX], jeDF[cU][target_field], jeDF[cV][target_field]
                elif i2 < j2 < t2 < k2: jeXT, jeUT, jeVT = jeDF[target_field][cX], jeDF[target_field][cU], jeDF[cV][target_field]
                elif i2 < j2 < k2 < t2: jeXT, jeUT, jeVT = jeDF[target_field][cX], jeDF[target_field][cU], jeDF[target_field][cV]
                qmi = eX+eU+eV+eT-jeXU-jeXV-jeXT-jeUV-jeUT-jeVT+jeXUV+jeXUT+jeXVT+jeUVT-jeXUVT
                qmiL.append(qmi)
                cxcucvL.append((cX,cU,cV))
    return pd.Series(qmiL, index=cxcucvL)

In [ ]:
def get_stepped_cE_data_for_DF(DF, cDF=None, step_count=MAX_STEP_COUNT):
    
    # FOR EACH COLUMN GET SORTED LIST OF OTHER COLUMNS IN ORDER OF DECREASING MUTUAL INFORMATION
    eDF = get_Es_for_DF(DF)
    e2DF = get_Es_for_DF(DF, cDF=cDF) if (cDF is not None) else None
    miDF = get_2v_MIs_for_DF(DF, cDF).fillna(0)
    miDFm = miDF + miDF.T    
    cDc2NmiL = {c: sorted(list(miDFm[c].drop(c).items()), reverse=True, key=lambda cNmi: abs(cNmi[1])) for c in miDFm.columns}
    
    # FOR EACH COLUMN GET step_count LONG LIST OF DECREASING JOINT ENTROPIES, INCREASING CONDITIONAL ENTROPIES
    cDc2NmiNceiL, ctDje = {}, {}
    for c in DF.columns:
        cDc2NmiNceiL[c], e = [], eDF[c]
        e2 = e2DF[c] if e2DF is not None else None
        for i in range(len(cDc2NmiL[c])):
            cL = [cNmi[0] for cNmi in cDc2NmiL[c][:i+1]]
            cT1, cT2 = tuple(sorted(cL+[c])), tuple(sorted(cL))
            if e2 is None:                
                je1=ctDje[cT1]=ctDje[cT1] if cT1 in ctDje else get_jE_for_DF(DF[list(cT1)])
                je2=ctDje[cT2]=ctDje[cT2] if cT2 in ctDje else get_jE_for_DF(DF[list(cT2)])
            else:
                je1=ctDje[cT1]=ctDje[cT1] if cT1 in ctDje else \
                        get_jE_for_DF(DF[list(cT1)], cDF[list(cT1)])
                je2=ctDje[cT2]=ctDje[cT2] if cT2 in ctDje else \
                        get_jE_for_DF(DF[list(cT2)], cDF[list(cT2)])
            cei = e-(je1-je2) if e2 is None else e2-(je1-je2)
            c2, mi = cDc2NmiL[c][i]
            if not cDc2NmiNceiL[c] or cei > cDc2NmiNceiL[c][-1][2]:
                cDc2NmiNceiL[c].append((c2, mi, cei))
            if len(cDc2NmiNceiL[c]) == step_count:    
                break
        if len(cDc2NmiNceiL[c]) != step_count:
            cDc2NmiNceiL[c].extend([cDc2NmiNceiL[c][-1]]*(step_count-len(cDc2NmiNceiL[c])))
        cDc2NmiNceiL[c] = cDc2NmiNceiL[c][::-1]
                
    # RETURN DATAFRAME OF max_count INCREASING CONDITIONAL ENTROPIES FOR EACH COLUMN
    return pd.DataFrame(cDc2NmiNceiL, columns=DF.columns).T

def get_stepped_cMI_data_for_DF_and_target_field(DF, target_field, cDF=None, 
            step_count=MAX_STEP_COUNT):
    miSS = get_2v_MIs_for_DF_and_target_field(DF, target_field, cDF=cDF)
    tmiDF = get_3v_MIs_for_DF_and_target_field(DF, target_field, cDF=cDF).fillna(0)
    cDc2NtmiNtmisL, tmiDFm = {}, tmiDF + tmiDF.T
    for c in tmiDFm.columns:
        c2NtmiL, miXY = list(tmiDFm[c].drop(c).items()), miSS[c]
        p_c2NtmiNatmiL =[(c2, tmi, tmi) for c2, tmi in c2NtmiL if tmi >= 0.0]
        p_c2NtmiNatmiL.sort(key=lambda c2NtmiNatmi: c2NtmiNatmi[2], reverse=True)
        n_c2NtmiNatmiL =[(c2, tmi, -tmi) for c2, tmi in c2NtmiL if tmi < 0.0]
        n_c2NtmiNatmiL.sort(key=lambda c2NtmiNatmi: c2NtmiNatmi[2], reverse=True)
        cDc2NtmiNtmisL[c] = [(None, 0.0, 0.0)]
        for c2, tmi, atmi in p_c2NtmiNatmiL+n_c2NtmiNatmiL:
            if cDc2NtmiNtmisL[c][-1][2]+atmi < miXY:
                cDc2NtmiNtmisL[c].append((c2, tmi, cDc2NtmiNtmisL[c][-1][2]+atmi))
            else:
                cDc2NtmiNtmisL[c].append((c2, tmi, miXY))
                break
        cDc2NtmiNtmisL[c] = cDc2NtmiNtmisL[c][1:]
        if len(cDc2NtmiNtmisL[c]) != step_count:
            cDc2NtmiNtmisL[c].extend([cDc2NtmiNtmisL[c][-1]]*(step_count-len(cDc2NtmiNtmisL[c])))
        cDc2NtmiNtmisL[c] = cDc2NtmiNtmisL[c][:step_count][::-1]
    return pd.DataFrame(cDc2NtmiNtmisL, columns=tmiDF.columns).T

### Test control functions...

In [ ]:
def _shuffle_DF(DF):
    D = {}
    for c, L in [(c, list(DF[c])) for c in DF.columns]:
        random.shuffle(L)
        D[c] = L
    return pd.DataFrame(D, DF.index)

def _synthesize_by_field_DF(DF):
    D = {}
    for c, SS in [(c, DF[c]) for c in DF.columns]:
        kT, cT = zip(*collect.Counter(SS).items())
        D[c] = random.choices(kT, weights=cT, k=SS.size)        
        #D[c] = random.choices(kT, k=SS.size)
    return pd.DataFrame(D, DF.index)

def _join_split_DFs(DF1, DF2):
    c1, c2, DF1m, DF2m = DF1.shape[0], DF2.shape[0], DF1, DF2
    if c1 < c2:
        m, b = c2 // c1, c2 % c1
        DF1m = pd.concat([DF1]*m + [DF1.iloc[range(b)]])  
    elif c2 < c1:
        m, b = c1 // c2, c1 % c2
        DF2m = pd.concat([DF2]*m + [DF2.iloc[range(b)]])    
    jDF = pd.concat([DF1m, DF2m])    
    
    rSS = pd.Series([random.choice([True, False]) for i in range(jDF.shape[0])], index=jDF.index)
    jsDF1, jsDF2 = jDF.loc[rSS], jDF.loc[~rSS]
    jsDF1.index, jsDF2.index = range(jsDF1.shape[0]), range(jsDF2.shape[0])
    return jsDF1, jsDF2

def _scramble_synthesize_DFs(DF1, DF2):    
    C1 = collect.Counter([tuple(vL) for vL in DF1.values.tolist()])
    C2 = collect.Counter([tuple(vL) for vL in DF2.values.tolist()])
    kL, s1, s2 = list(set(C1.keys()) | set(C2.keys())), sum(C1.values()), sum(C2.values()) 
    pL = [(C1[k]/s1 + C2[k]/s2)/2.0 for k in kL]
    L1 = random.choices(kL, weights=pL, k=max(DF1.size, DF2.size))
    L2 = random.choices(kL, weights=pL, k=max(DF1.size, DF2.size))
    return pd.DataFrame(L1, columns=DF1.columns), pd.DataFrame(L2, columns=DF1.columns)
  
def _filter_outliers(SS):
    q1, q2, q3 = SS.quantile([.25, .5, .75])
    bSS = (q2-(q3-q1)*1.5 <= SS) & (SS < q2+(q3-q1)*1.5)
    return SS[bSS], SS[~bSS]
        
def _get_outlier_filtered_mean_and_stdev(SS, mirror=False):
    SS2 = pd.Series(list(np.abs(SS))+list(-np.abs(SS))) if mirror else SS
    fSS, _ = _filter_outliers(SS2)
    return np.mean(fSS), np.std(fSS)

def get_filtered_linear_least_squares_fit(SSx, SSy):
    bSS = np.abs(SSy/SSx) <= 100.0
    #fSSx, fSSy = SSx[bSS], SSy[bSS]
    fSSxy, oSSxy = np.sqrt(SSx**2+SSy**2)[bSS].sort_values(), np.sqrt(SSx**2+SSy**2)[~bSS]
    fSSxysl = fSSxy
    #fSSxy2, oSSxy2 = fSSxy[fSSxy.size//2:], fSSxy[:fSSxy.size//2]
    #fSSxysl, oSSsl = _filter_outliers(SSy[fSSxy2.index]/SSx[fSSxy2.index])
    b, m = polyfit(pd.concat([SSx[fSSxysl.index], -SSx[fSSxysl.index]]), 
                   pd.concat([SSy[fSSxysl.index], -SSy[fSSxysl.index]]), 1)
    return b, m, list(fSSxysl.index), list(oSSxy.index)#+list(oSSxy2.index)+list(oSSsl.index)


def get_filtered_linear_least_squares_fit2(SSx, SSy):
    #SSxy = np.sqrt(SSx**2+SSy**2).sort_values()
    #fSSxy, oSSxy = SSxy[SSxy.size//2:], SSxy[:SSxy.size//2]
    sSSx = SSx.sort_values()
    fSSx, oSSx = sSSx[SSx.size//2:], sSSx[:SSx.size//2]
    fSSxsl, oSSsl = _filter_outliers(SSy[fSSx.index]/SSx[fSSx.index])
    #b, m = polyfit(SSx[fSSsl.index], SSy[fSSsl.index], 1)
    b, m = polyfit(pd.concat([SSx[fSSxsl.index], -SSx[fSSxsl.index]]), 
                   pd.concat([SSy[fSSxsl.index], -SSy[fSSxsl.index]]), 1)
    return b, m, list(fSSxsl.index), list(oSSx.index)+list(oSSsl.index)

def _get_Z(SS, dSS=None, mFlag=None):
    zSS, dSS = None, (SS if dSS is None else dSS)
    if mFlag is None:
        mn, std = _get_outlier_filtered_mean_and_stdev(dSS)
        zSS = (SS-mn)/std
    elif mFlag == 'mirror': 
        mn, std = _get_outlier_filtered_mean_and_stdev(dSS, mirror=True)
        zSS = (SS-mn)/std
    elif mFlag == 'dmirror':
        mn, std = _get_outlier_filtered_mean_and_stdev(dSS[dSS>=0], mirror=True)
        mn2, std2 = _get_outlier_filtered_mean_and_stdev(dSS[dSS<0], mirror=True)
        zSS = pd.Series([((v-mn)/std if v >= 0 else (v-mn2)/std2) for v in SS], index=SS.index)
    else: raise AssertionError('mFlag is not recognized')
    return zSS

In [ ]:
def test(tDF, cDF):
    tDFrs, cDFrs = _synthesize_by_field_DF(tDF), _synthesize_by_field_DF(cDF)
    tDFjs, cDFjs = _join_split_DFs(tDF, cDF)    

    miSStrs, miSScrs, miSSxrs, miSStjs, miSScjs, miSSxjs = None, None, None, None, None, None
    if True:
        miDFt, miDFc = get_2v_MIs_for_DF(tDF), get_2v_MIs_for_DF(cDF)
        miDFx = get_2v_MIs_for_DF(tDF, cDF)
        
        miDFtmx, miDFcmx = get_max_2v_MIs_for_DF(tDF), get_max_2v_MIs_for_DF(cDF)
        
        miDFtrs, miDFcrs = get_2v_MIs_for_DF(tDFrs), get_2v_MIs_for_DF(cDFrs)
        miDFxrs = get_2v_MIs_for_DF(tDFrs, cDFrs)
        miDFtjs, miDFcjs = get_2v_MIs_for_DF(tDFjs), get_2v_MIs_for_DF(cDFjs)
        miDFxjs = get_2v_MIs_for_DF(tDFjs, cDFjs)
        icL = [(i, c) for c in miDFt for i in miDFt.index if not pd.isna(miDFt[c][i])]
        miSSt = pd.Series([v for c in miDFt for v in miDFt[c] if not pd.isna(v)], index=icL)
        miSSc = pd.Series([v for c in miDFc for v in miDFc[c] if not pd.isna(v)], index=icL)
        miSSx = pd.Series([v for c in miDFx for v in miDFx[c] if not pd.isna(v)], index=icL)
        
        miSStmx = pd.Series([v for c in miDFtmx for v in miDFtmx[c] if not pd.isna(v)], index=icL)
        miSScmx = pd.Series([v for c in miDFcmx for v in miDFcmx[c] if not pd.isna(v)], index=icL)
        
        miSStrs = pd.Series([v for c in miDFtrs for v in miDFtrs[c] if not pd.isna(v)], index=icL)
        miSScrs = pd.Series([v for c in miDFcrs for v in miDFcrs[c] if not pd.isna(v)], index=icL)
        miSSxrs = pd.Series([v for c in miDFxrs for v in miDFxrs[c] if not pd.isna(v)], index=icL)
        miSStjs = pd.Series([v for c in miDFtjs for v in miDFtjs[c] if not pd.isna(v)], index=icL)
        miSScjs = pd.Series([v for c in miDFcjs for v in miDFcjs[c] if not pd.isna(v)], index=icL)
        miSSxjs = pd.Series([v for c in miDFxjs for v in miDFxjs[c] if not pd.isna(v)], index=icL)
    else:
        miSSt, miSSc = get_3v_MIs_for_DF(tDF), get_3v_MIs_for_DF(cDF)
        miSSx = get_3v_MIs_for_DF(tDF, cDF)
        
        miSStmx, miSScmx = get_max_3v_MIs_for_DF(tDF), get_max_3v_MIs_for_DF(cDF)
        
        miSStrs, miSScrs = get_3v_MIs_for_DF(tDFrs), get_3v_MIs_for_DF(cDFrs)
        miSSxrs = get_3v_MIs_for_DF(tDFrs, cDFrs)
        miSStjs, miSScjs = get_3v_MIs_for_DF(tDFjs), get_3v_MIs_for_DF(cDFjs)
        miSSxjs = get_3v_MIs_for_DF(tDFjs, cDFjs)
        
        
    with plt.rc_context({'xtick.color': 'white', 'ytick.color': 'white', 'text.color': 'white'}):
        plt.figure(1, figsize=(12, 18))

        
        plt.subplot(3, 2, 1)
        plt.title('Non-normalized Test vs Control')

        #plt.plot(miSStjs, 0.0+1.0*miSStjs, '-', color='green', alpha=0.5)
        b1, m1, fL1, oL1 = get_filtered_linear_least_squares_fit(miSSt, miSSc)
        plt.plot(miSSt, b1+m1*miSSt, '-', color='blue', alpha=0.5)
        #b1js, m1js, fL1js, oL1js = get_filtered_linear_least_squares_fit(miSStjs, miSScjs)
        #plt.plot(miSStjs, b1js+m1js*miSStjs, '-', color='red', alpha=0.5)

        plt.scatter(miSSt, miSSc, marker='.', alpha=0.5, 
                    c=pd.Series(range(miSSt.size))/miSSt.size, cmap=plt.get_cmap('tab20'))        
        
        plt.scatter(miSSt[oL1], miSSc[oL1], marker='.', alpha=1.0, color='red')

        
        plt.subplot(3, 2, 2)
        plt.title('Non-normalized Test vs Cross')
        
        #lt.plot(miSStjs, 0.0+1.0*miSStjs, '-', color='green', alpha=0.5)
        b2, m2, fL2, oL2 = get_filtered_linear_least_squares_fit(miSSt, miSSx)
        plt.plot(miSSt, b2+m2*miSSt, '-', color='blue', alpha=0.5)
        #2js, m2js, fL2js, oL2js = get_filtered_linear_least_squares_fit(miSStjs, miSSxjs)
        #lt.plot(miSStjs, b2js+m2js*miSStjs, '-', color='red', alpha=0.5)

        plt.scatter(miSSt, miSSx, marker='.', alpha=0.5, 
                    c=pd.Series(range(miSSt.size))/miSSt.size, cmap=plt.get_cmap('tab20'))
        plt.scatter(miSSt[oL2], miSSx[oL2], marker='.', alpha=1.0, color='red')

        plt.subplot(3, 2, 3)
        plt.title('Non-normalized Test vs Control (normalized)')
        
        plt.scatter(miSSt/miSStmx, miSSc/miSScmx, marker='.', alpha=0.5, 
                    c=pd.Series(range(miSSt.size))/miSSt.size, cmap=plt.get_cmap('tab20'))        
        
        plt.subplot(3, 2, 4)
        plt.title('Non-normalized Test vs Cross (normalized)')
        
        plt.scatter(miSSt/miSStmx, miSSx, marker='.', alpha=0.5, 
                    c=pd.Series(range(miSSt.size))/miSSt.size, cmap=plt.get_cmap('tab20'))
        
        plt.subplot(3, 2, 5)
        plt.title('Non-normalized Test vs Control (filtered)')
        
        #plt.plot(miSStjs, b1js+m1js*miSStjs, '-', color='red', alpha=0.5)
        plt.plot(miSSt, b1+m1*miSSt, '-', color='blue', alpha=0.5)
        
        plt.scatter(miSSt, miSSc, marker='.', alpha=0.5, 
                    c=pd.Series(range(miSSt.size))/miSSt.size, cmap=plt.get_cmap('tab20'))
        plt.scatter(miSStrs, miSScrs, marker='+', alpha=1.0, color='black')        
        plt.scatter(miSStjs, miSScjs, marker='+', alpha=1.0, color='gray')
        
        
        plt.subplot(3, 2, 6)
        plt.title('Non-normalized Test vs Cross (filtered)')        
        
        #plt.plot(miSStjs, b2js+m2js*miSStjs, '-', color='red', alpha=0.5)
        plt.plot(miSSt, b2+m2*miSSt, '-', color='blue', alpha=0.5)

        plt.scatter(miSSt, miSSx, marker='.', alpha=0.5,
                    c=pd.Series(range(miSSt.size))/miSSt.size, cmap=plt.get_cmap('tab20'))
        plt.scatter(miSStrs, miSSxrs, marker='+', alpha=1.0, color='black')        
        plt.scatter(miSStjs, miSSxjs, marker='+', alpha=1.0, color='gray')
        
    dSS1 = None
    return dSS1
dSS = test(eDF, eDFc)
#print(dSS)

### Test control functions 2...

In [ ]:
def _get_0norm_d_scores(SS):
    return SS
def _get_0norm_fd_scores(SS, window=5):
    vL = [np.NaN]*window//2
    vL += [sum(SS[i-window:i]) for i in range(window, SS.size)]        
    vL += [np.NaN]*window//2
    return pd.Series(nL, index=SS.index)

def _get_0norm_k_scores(SS):
    return SS

def _get_0norm_filtered_SS(SS, 
                           miSScr, miSStr, miSSxr,
                           miSScjs, miSStjs, miSSxjs):
    sSS, cL = SS.sort_values(), ['v','vZ','vZd','vZdZ','vZdZp','kZ','kZp']
    iL, vNvzNvzdNvzdzNvzdzpNkzNkzpL = [], []
    for i, i2 in enumerate(range(10, sSS.size)):
        mSS = pd.concat([sSS[:i2], -sSS[:i2]])
        v, std, (kz, kzp) = sSS[i], np.std(mSS), stats.kurtosistest(mSS)
        vz, vzd = v/std, (v/std-vNvzNvzdNvzdzNvzdzpNkzNkzpL[i-1][1]) if (i > 0) else 0.0
        iL.append(i2)
        vNvzNvzdNvzdzNvzdzpNkzNkzpL.append((v, v/std, vzd, np.NaN, np.NaN, kz, kzp))
    sDF = pd.DataFrame(vNvzNvzdNvzdzNvzdzpNkzNkzpL, index=iL, columns=cL)
    zSS = pd.Series((sDF['vZd']-sDF['vZd'].mean())/sDF['vZd'].std(), index=iL)
    pSS = pd.Series([(stats.norm.cdf(z) if not np.isnan(z) else np.NaN) for z in zSS], index=iL)
    sDF['vZdZ'], sDF['vZdZp'] = zSS, pSS
    
    print(sDF)

    i = sSS.size
    scSS1 = _get_0norm_d_scores(sDF['vZ'])
    scSS2 = _get_0norm_fd_scores(sDF['vZd'])
    scSS3 = _get_0norm_k_scores(sDF['kZ'])
    scSS = np.sqrt(scSS1**2+scSS2**2+scSS3**2)
    
#     if zNiL: i = ([zNiL[-1][1]+1]+[dNi[1] for dNi in dNiL if dNi[1] <= zNiL[-1][1]+1])[-1]
#     elif dNiL: i = ([dNi[1] for dNi in dNiL])[-1] 

    if True:
        with plt.rc_context({'xtick.color': 'white', 'ytick.color': 'white', 'text.color': 'white'}):
            plt.figure(1, figsize=(18, 18))
            plt.subplot(3, 3, 1)
            plt.title('stdevs of d from 0')
            plt.hist(sDF['vZ'], 20, color='red')
            plt.subplot(3, 3, 2)
            plt.title("stdevs from ave f'(d)")
            plt.hist(sDF['vZdZ'], 20, color='green')
            plt.subplot(3, 3, 3)
            plt.title("kurtosis")
            plt.hist(sDF['kZ'], 20, color='blue')

            plt.subplot(3, 3, 4)
            plt.title('stdevs of d from 0')
            plt.scatter(sDF.index, sDF['vZ'], marker='.', color='red')
            plt.subplot(3, 3, 5)
            plt.title("stdevs from ave f'(d)")
            plt.scatter(sDF.index, sDF['vZdZ'], marker='.', color='green')
            plt.subplot(3, 3, 6)
            plt.title("kurtosis")
            plt.scatter(sDF.index, sDF['kZ'], marker='.', color='blue')

            plt.subplot(3, 3, 8)      
            plt.title('Non-normalized Test vs Control')
            plt.scatter(miSStjs, miSScjs, marker='.', alpha=0.1, color='gray')
            plt.scatter(miSStr, miSScr, marker='.', alpha=0.5, color='black')
            plt.scatter(miSStr[sSS[:i].index], miSScr[sSS[:i].index], 
                        marker='.', alpha=1.0, color='blue')

            plt.subplot(3, 3, 9)
            plt.title('Non-normalized Test vs Cross-Entropy')
            plt.scatter(miSStjs, miSSxjs, marker='.', alpha=0.1, color='gray')
            plt.scatter(miSStr, miSSxr, marker='.', alpha=0.5, color='black')
            plt.scatter(miSStr[sSS[:i].index], miSSxr[sSS[:i].index], 
                        marker='.', alpha=1.0, color='blue')

    return sSS[:i]

def test(tDF, cDF):
    tDFr, cDFr = _synthesize_by_field_DF(tDF), _synthesize_by_field_DF(cDF)
    tDFjs, cDFjs = _join_split_DFs(tDF, cDF)    

    miSStr, miSScr, miSSxr = None, None, None
    if True:
        miDFtr, miDFcr = get_2v_MIs_for_DF(tDFr), get_2v_MIs_for_DF(cDFr)
        miDFxr = get_2v_MIs_for_DF(tDFr, cDFr)
        miDFtjs, miDFcjs = get_2v_MIs_for_DF(tDFjs), get_2v_MIs_for_DF(cDFjs)
        miDFxjs = get_2v_MIs_for_DF(tDFjs, cDFjs)
        icL = [(i, c) for c in miDFtr for i in miDFtr.index if not pd.isna(miDFtr[c][i])]
        miSStr = pd.Series([v for c in miDFtr for v in miDFtr[c] if not pd.isna(v)], index=icL)
        miSScr = pd.Series([v for c in miDFcr for v in miDFcr[c] if not pd.isna(v)], index=icL)
        miSSxr = pd.Series([v for c in miDFxr for v in miDFxr[c] if not pd.isna(v)], index=icL)
        miSStjs = pd.Series([v for c in miDFtjs for v in miDFtjs[c] if not pd.isna(v)], index=icL)
        miSScjs = pd.Series([v for c in miDFcjs for v in miDFcjs[c] if not pd.isna(v)], index=icL)
        miSSxjs = pd.Series([v for c in miDFxjs for v in miDFxjs[c] if not pd.isna(v)], index=icL)
    if False:
        miSStr, miSScr = get_3v_MIs_for_DF(tDFr), get_3v_MIs_for_DF(cDFr)
        miSSxr = get_3v_MIs_for_DF(tDFr, cDFr)
        miSStjs, miSScjs = get_3v_MIs_for_DF(tDFjs), get_3v_MIs_for_DF(cDFjs)
        miSSxjs = get_3v_MIs_for_DF(tDFjs, cDFjs)

    dSS_mi0n = _get_0norm_filtered_SS(np.sqrt(miSStr**2+miSScr**2), 
                                      miSStr, miSScr, miSSxr, 
                                      miSStjs, miSScjs, miSSxjs)
    return dSS_mi0n
dSS = test(eDF, eDFc)

In [ ]:
def get_Z_of_2v_MIs(tDF, cDF):
        
    miDFt, miDFc = get_2v_MIs_for_DF(tDF), get_2v_MIs_for_DF(cDF)
    miDFx = get_2v_MIs_for_DF(tDF, cDF)
        
    tDFr, cDFr = _synthesize_by_field_DF(tDF), _synthesize_by_field_DF(cDF)
    miDFtr, miDFcr = get_2v_MIs_for_DF(tDFr), get_2v_MIs_for_DF(cDFr)
    miDFxr = get_2v_MIs_for_DF(tDFr, cDFr)

    tDFjs, cDFjs = _join_split_DFs(tDF, cDF)    
    miDFtjs, miDFcjs = get_2v_MIs_for_DF(tDFjs), get_2v_MIs_for_DF(cDFjs)
    miDFxjs = get_2v_MIs_for_DF(tDFjs, cDFjs)

    icL = [(i, c) for c in miDFt for i in miDFt.index if not pd.isna(miDFt[c][i])]
    miSSt = pd.Series([v for c in miDFt for v in miDFt[c] if not pd.isna(v)], index=icL)
    miSSc = pd.Series([v for c in miDFc for v in miDFc[c] if not pd.isna(v)], index=icL)
    miSSx = pd.Series([v for c in miDFx for v in miDFx[c] if not pd.isna(v)], index=icL)
    miSStr = pd.Series([v for c in miDFtr for v in miDFtr[c] if not pd.isna(v)], index=icL)
    miSScr = pd.Series([v for c in miDFcr for v in miDFcr[c] if not pd.isna(v)], index=icL)
    miSSxr = pd.Series([v for c in miDFxr for v in miDFxr[c] if not pd.isna(v)], index=icL)
    miSStjs = pd.Series([v for c in miDFtjs for v in miDFtjs[c] if not pd.isna(v)], index=icL)
    miSScjs = pd.Series([v for c in miDFcjs for v in miDFcjs[c] if not pd.isna(v)], index=icL)
    miSSxjs = pd.Series([v for c in miDFxjs for v in miDFxjs[c] if not pd.isna(v)], index=icL)

    zSSt, zSSc = _get_Z(miSSt, mFlag='mirror'), _get_Z(miSSc, mFlag='mirror')
    zSSx = _get_Z(miSSx, mFlag='mirror')

    zSStr, zSScr = _get_Z(miSStr, miSSt, mFlag='mirror'), _get_Z(miSScr, miSSc, mFlag='mirror')
    zSSxr = _get_Z(miSSxr, miSSx, mFlag='mirror')

    zSStjs, zSScjs = _get_Z(miSStjs, mFlag='mirror'), _get_Z(miSScjs, mFlag='mirror')
    zSSxjs = _get_Z(miSSxjs, mFlag='mirror')

    
    # DERIVATIVE MI CALCULATIONS
    
    dSS_mi0n = _get_0norm_filtered_SS(np.sqrt(miSStr**2+miSScr**2))
    std_mi0n = np.std(pd.concat([dSS_mi0n, -dSS_mi0n]))
    zSStc_mi0n = np.sqrt(miSSt**2+miSSc**2)/std_mi0n
    
    dSS_z0n = _get_0norm_filtered_SS(np.sqrt(zSStr**2+zSScr**2))
    std_z0n = np.std(pd.concat([dSS_z0n, -dSS_z0n]))
    zSStc_z0n = np.sqrt(zSSt**2+zSSc**2)/std_z0n
    
    _process_Z_stuff(miSSt, miSSc, miSSx, miSStr, miSScr, miSSxr, miSStjs, miSScjs, miSSxjs, 
                     zSSt, zSSc, zSSx, zSStr, zSScr, zSSxr, zSStjs, zSScjs, zSSxjs,
                     zSStc_mi0n, zSStc_z0n, dSS_mi0n, dSS_z0n)
    #tD = {c: [np.NaN]*miDFt.shape[1] for c in miDFt.columns}
    #for i, c in icL: tD[c][miDFt.index.get_loc(i)] = tSS[(i, c)]
    #zDF = pd.DataFrame(tD, miDFt.index)
    return None

def get_Z_of_3v_MIs(tDF, cDF):
        
    miSSt, miSSc = get_3v_MIs_for_DF(tDF), get_3v_MIs_for_DF(cDF) 
    miSSx = get_3v_MIs_for_DF(tDF, cDF)

    tDFr, cDFr = _synthesize_by_field_DF(tDF), _synthesize_by_field_DF(cDF)
    miSStr, miSScr = get_3v_MIs_for_DF(tDFr), get_3v_MIs_for_DF(cDFr)
    miSSxr = get_3v_MIs_for_DF(tDFr, cDFr)

    tDFjs, cDFjs = _join_split_DFs(tDF, cDF)
    miSStjs, miSScjs = get_3v_MIs_for_DF(tDFjs), get_3v_MIs_for_DF(cDFjs)
    miSSxjs = get_3v_MIs_for_DF(tDFjs, cDFjs)
    
    zSSt, zSSc = _get_Z(miSSt, mFlag='mirror'),  _get_Z(miSSc, mFlag='mirror')
    zSSx = _get_Z(-miSSx, mFlag='mirror')
    
    zSStr, zSScr = _get_Z(miSStr, miSSt, mFlag='mirror'),  _get_Z(miSScr, miSSc, mFlag='mirror')
    zSSxr = _get_Z(-miSSxr, -miSSx, mFlag='mirror')

    zSStjs, zSScjs = _get_Z(miSStjs, miSSt, mFlag='mirror'),  _get_Z(miSScjs, miSSc, mFlag='mirror')
    zSSxjs = _get_Z(-miSSxjs, -miSSx, mFlag='mirror')
    
    
    # DERIVATIVE MI CALCULATIONS
    
    dSS_mi0n = _get_0norm_filtered_SS(np.sqrt(miSStr**2+miSScr**2))
    std_mi0n = np.std(pd.concat([dSS_mi0n, -dSS_mi0n]))
    zSStc_mi0n = np.sqrt(miSSt**2+miSSc**2)/std_mi0n
    
    dSS_z0n = _get_0norm_filtered_SS(np.sqrt(zSStr**2+zSScr**2))
    std_z0n = np.std(pd.concat([dSS_z0n, -dSS_z0n]))
    zSStc_z0n = np.sqrt(zSSt**2+zSSc**2)/std_z0n
    
    _process_Z_stuff(miSSt, miSSc, -miSSx, miSStr, miSScr, -miSSxr, miSStjs, miSScjs, -miSSxjs, 
                     zSSt, zSSc, zSSx, zSStr, zSScr, zSSxr, zSStjs, zSScjs, zSSxjs,
                     zSStc_mi0n, zSStc_z0n, dSS_mi0n, dSS_z0n)
    return None

In [ ]:
PRINT_PLOTS = True
PRINT_TABLE = False
    
def _process_Z_stuff(miSSt, miSSc, miSSx, miSStr, miSScr, miSSxr, miSStjs, miSScjs, miSSxjs, 
                     zSSt, zSSc, zSSx, zSStr, zSScr, zSSxr, zSStjs, zSScjs, zSSxjs,
                     zSStc_mi0n, zSStc_z0n, dSS_mi0n, dSS_z0n):
    
    if PRINT_PLOTS:
                
        fSStc_mi0n = zSStc_mi0n < 2
        fSStc_z0n = zSStc_z0n < 2
        
        mn_mitr, mn_micr = np.mean(miSStr), np.mean(miSScr) 
        mn_ztr, mn_zcr = np.mean(zSStr), np.mean(zSScr)            
        b_mitc, m_mitc = polyfit(miSStjs, miSScjs, 1)            
        b_mitx, m_mitx = polyfit(miSStjs, miSSxjs, 1)
        b_ztc, m_ztc = polyfit(zSStjs, zSScjs, 1)            
        b_ztx, m_ztx = polyfit(zSStjs, zSSxjs, 1)

        with plt.rc_context({'xtick.color': 'white', 'ytick.color': 'white', 'text.color': 'white'}):
            plt.figure(1, figsize=(18, 30))

            ####################################
            # NON-NORMALIZED PLOTS

            plt.subplot(5, 3, 1)      
            plt.title('Non-normalized Test vs Control')
            plt.plot(miSSt, b_mitc+m_mitc*miSSt, '-', alpha=0.1, color='red')
            plt.scatter(miSStjs, miSScjs, marker='.', alpha=0.1, color='gray')
            plt.scatter(miSStr, miSScr, marker='.', alpha=0.5, color='black')
            plt.scatter(miSStr[dSS_mi0n.index], miSScr[dSS_mi0n.index], 
                        marker='.', alpha=1.0, color='blue')

#             plt.scatter(miSSt, miSSc, marker='.', alpha=1.0, color='orange')
#             for ni, i in enumerate(miSSt[~fSStcr].index):
#                 tv, cv =  miSSt[i], miSSc[i]
#                 tvjs, cvjs =  miSStjs[i], miSScjs[i]
#                 plt.plot([tvjs, tv], [cvjs, cv], '-',  
#                          color=plt.get_cmap('tab20')(ni/miSSt[~fSStcr].size), alpha=0.5)
#             plt.scatter(miSSt[~fSStcr], miSSc[~fSStcr], marker='.',
#                         c=pd.Series(range(miSSt[~fSStcr].size))/miSSt[~fSStcr].size, 
#                         cmap=plt.get_cmap('tab20'), alpha=1.0)

            plt.subplot(5, 3, 2) 
            plt.title('Non-normalized Test vs Cross-Entropy')        
            plt.plot(miSSt, b_mitx+m_mitx*miSSt, '-', alpha=0.1, color='red')
            plt.scatter(miSStjs, miSSxjs, marker='.', alpha=0.1, color='gray')
            plt.scatter(miSStr, miSSxr, marker='.', alpha=0.5, color='black')
            plt.scatter(miSStr[dSS_mi0n.index], miSSxr[dSS_mi0n.index], 
                        marker='.', alpha=1.0, color='blue')

#             plt.scatter(miSSt, miSSx, marker='.', alpha=1.0, color='orange')
#             for ni, i in enumerate(miSSt[~fSStcr].index):
#                 tv, xv =  miSSt[i], miSSx[i]
#                 tvjs, xvjs =  miSStjs[i], miSSxjs[i]
#                 plt.plot([tvjs, tv], [xvjs, xv], '-',  
#                          color=plt.get_cmap('tab20')(ni/miSSt[~fSStcr].size), alpha=0.5)
#             plt.scatter(miSSt[~fSStcr], miSSx[~fSStcr], marker='.',
#                         c=pd.Series(range(miSSt[~fSStcr].size))/miSSt[~fSStcr].size, 
#                         cmap=plt.get_cmap('tab20'), alpha=1.0)


            ####################################
            # NORMALIZED PLOTS

            plt.subplot(5, 3, 4)       
            plt.title('Normalized Test vs Control')
            plt.plot(zSSt, b_ztc+m_ztc*zSSt, '-', alpha=0.1, color='red')
            plt.scatter(zSStjs, zSScjs, marker='.', alpha=0.1, color='gray')
            plt.scatter(zSStr, zSScr, marker='.', alpha=0.5, color='black')
            plt.scatter(zSStr[dSS_z0n.index], zSScr[dSS_z0n.index], 
                        marker='.', alpha=1.0, color='blue')

#             plt.scatter(zSSt, zSSc, marker='.', alpha=1.0, color='orange')1
#             for ni, i in enumerate(zSSt[~fSStcr2].index):
#                 tv, cv =  zSSt[i], zSSc[i]
#                 tvjs, cvjs =  zSStjs[i], zSScjs[i]
#                 plt.plot([tvjs, tv], [cvjs, cv], '-',  
#                          color=plt.get_cmap('tab20')(ni/zSSt[~fSStcr2].size), alpha=0.5)
#             plt.scatter(zSSt[~fSStcr2], zSSc[~fSStcr2], marker='.',
#                         c=pd.Series(range(zSSt[~fSStcr2].size))/zSSt[~fSStcr2].size, 
#                         cmap=plt.get_cmap('tab20'), alpha=1.0)
            
            plt.subplot(5, 3, 5)
            plt.title('Normalized Test vs Cross-Entropy')
            plt.plot(zSSt, b_ztx+m_ztx*zSSt, '-', alpha=0.1, color='red')
            plt.scatter(zSStjs, zSSxjs, marker='.', alpha=0.1, color='gray')
            plt.scatter(zSStr, zSSxr, marker='.', alpha=0.5, color='black')
            plt.scatter(zSStr[dSS_z0n.index], zSSxr[dSS_z0n.index], 
                        marker='.', alpha=1.0, color='blue')

#             plt.scatter(zSSt, zSSx, marker='.', alpha=1.0, color='orange')
#             for ni, i in enumerate(zSSt[~fSStcr2].index):
#                 tv, xv =  zSSt[i], zSSx[i]
#                 tvjs, xvjs =  zSStjs[i], zSSxjs[i]
#                 plt.plot([tvjs, tv], [xvjs, xv], '-',  
#                          color=plt.get_cmap('tab20')(ni/zSSt[~fSStcr2].size), alpha=0.5)
#             plt.scatter(zSSt[~fSStcr2], zSSx[~fSStcr2], marker='.',
#                         c=pd.Series(range(zSSt[~fSStcr2].size))/zSSt[~fSStcr2].size, 
#                         cmap=plt.get_cmap('tab20'), alpha=1.0)


            ###############################################
            # DISTRIBUTIONS OF DISTANCES FROM RANDOM 

            plt.subplot(5, 3, 7)
            plt.title('Test/control distribution relative to random')
            dSSr_mi =  np.sqrt(miSStr**2+miSScr**2)
            rSS = pd.Series([random.random() for i in range(miSSt.size)], index=miSSt.index)
            plt.scatter(dSSr_mi, rSS, marker='.', alpha=0.5, color='red')
            plt.scatter(dSSr_mi[dSS_mi0n.index], rSS[dSS_mi0n.index], 
                        marker='.', alpha=0.5, color='green')
          
            plt.subplot(5, 3, 8)
            plt.title('Test/control distribution relative to random/ave (hist)')
            plt.hist(dSSr_mi, 20, histtype='stepfilled', alpha=0.5, color='red')
            plt.hist(dSSr_mi[dSS_mi0n.index], 10, histtype='stepfilled', alpha=0.5, color='green')


#             #########################################################################
#             # DISTRIBUTIONS OF DISTANCES FROM SCRAMBLED SYNTHETIC (TEST V CONTROL)
            
#             plt.subplot(5, 3, 10)
#             plt.title('Test/control distribution relative to scrambled values')
#             dSStcjs = np.sqrt((miSSt-miSStjs)**2 + (miSSc-miSScjs)**2)
#             rSS = pd.Series([random.random() for i in range(miSSt.size)], index=miSSt.index)
#             plt.scatter(dSStcjs, rSS, marker='.', alpha=0.5,
#                         c=pd.Series(range(miSSt.size))/miSSt.size, cmap=plt.get_cmap('tab20'))

#             plt.subplot(5, 3, 11)
#             plt.title('Test/control distribution relative to scrambled ave')
#             dSStcjs2 = np.abs(-1*miSSt + m_mitc*miSSc + b_mitc)/np.sqrt(m_mitc**2 + 1)
#             dSStcjs3 = np.abs(-1*miSStjs + m_mitc*miSScjs + b_mitc)/np.sqrt(m_mitc**2 + 1)
#             rSS = pd.Series([random.random() for i in range(miSSt.size)], index=miSSt.index)
#             plt.scatter(dSStcjs2, rSS, marker='.', alpha=0.5,
#                         c=pd.Series(range(miSSt.size))/miSSt.size, cmap=plt.get_cmap('tab20'))
                        
#             plt.subplot(5, 3, 12)
#             plt.title('Test/control distribution relative to scrambled (hist)')
#             plt.hist(dSStcjs, 20, histtype='stepfilled', alpha=0.5, color='red')
#             plt.hist(dSStcjs2, 20, histtype='stepfilled', alpha=0.5, color='green')
#             plt.hist(dSStcjs3, 5, histtype='stepfilled', alpha=0.5, color='blue')
            
            
#             ##################################################################################
#             # DISTRIBUTIONS OF DISTANCES FROM SCRAMBLED SYNTHETIC (TEST VS CROSS-ENTROPY)
            
#             plt.subplot(5, 3, 13)
#             plt.title('Test/cross-entropy distribution relative to scrambled values')
#             dSStxjs = np.sqrt((miSSt-miSStjs)**2 + (miSSx-miSSxjs)**2)
#             rSS = pd.Series([random.random() for i in range(miSSt.size)], index=miSSt.index)
#             plt.scatter(dSStxjs, rSS, marker='.', alpha=0.5,
#                         c=pd.Series(range(miSSt.size))/miSSt.size, cmap=plt.get_cmap('tab20'))

#             plt.subplot(5, 3, 14)
#             plt.title('Test/cross-entropy distribution relative to scrambled ave')
#             dSStxjs2 = np.abs(-1*miSSt + m_mitx*miSSx + b_mitx)/np.sqrt(m_mitx**2 + 1)
#             dSStxjs3 = np.abs(-1*miSStjs + m_mitx*miSSxjs + b_mitx)/np.sqrt(m_mitx**2 + 1)
#             rSS = pd.Series([random.random() for i in range(miSSt.size)], index=miSSt.index)
#             plt.scatter(dSStxjs2, rSS, marker='.', alpha=0.5,
#                         c=pd.Series(range(miSSt.size))/miSSt.size, cmap=plt.get_cmap('tab20'))
            
#             plt.subplot(5, 3, 15)
#             plt.title('Test/cross-entropy distribution relative to scrambled (hist)')
#             plt.hist(dSStxjs, 20, histtype='stepfilled', alpha=0.5, color='red')
#             plt.hist(dSStxjs2, 20, histtype='stepfilled', alpha=0.5, color='green')
#             plt.hist(dSStxjs3, 5, histtype='stepfilled', alpha=0.5, color='blue')

    if PRINT_TABLE:
        pprint(pd.DataFrame({'zSStcr': zSStcr, 'zzSStcr': zzSStcr}))

    return None
    
get_Z_of_2v_MIs(eDF, eDFc)

In [ ]:
get_Z_of_3v_MIs(eDF, eDFc)

### Presentation constants and utility functions...

In [ ]:
def get_scale(total_E, target_radius):
    return target_radius/entropy2radii(total_E)

def get_color(color, cmap, vmin, vmax): 
    return cmap((float(color)-float(vmin))/(float(vmax)-float(vmin))) if isinstance(color, numbers.Number) else color

def get_graph_center_position(Graph, node_positions):
    npL = [node_positions[ni] for ni in Graph.nodes()]
    return sum([p[0] for p in npL])/len(npL), sum([p[1] for p in npL])/len(npL)

def get_max_radius(Graph, node_positions, center_position):
    dxdyL = [(center_position[0]-node_positions[ni][0], center_position[1]-node_positions[ni][1]) for ni in Graph.nodes()]
    return max([(dx**2 + dy**2)**0.5 for dx, dy in dxdyL])

def get_node_triplets(Graph):
    nodes, ntL = list(Graph.nodes()), []
    for i in range(len(nodes)):
        for j in range(i+1, len(nodes)):
            for k in range(j+1, len(nodes)):
                ntL.append((nodes[i], nodes[j], nodes[k]))
    return ntL

def entropy2radii(pdO):
    return (np.absolute(pdO)/math.pi)**0.5

def entropy2color(pdO):
    pdO2 = pdO.fillna(0.0)
    return np.sqrt(np.absolute(pdO2))*np.sign(pdO2)
    
def get_node_letters(Graph, node_positions):
    max_v = max(np.absolute([p[0] for p in node_positions.values()] + \
                            [p[1] for p in node_positions.values()]))    
    niNnpL = sorted([(ni, node_positions[ni]) for ni in Graph.nodes()], 
                    key=lambda niNnp: ((max_v-niNnp[1][0])**2+(niNnp[1][1]+max_v)**2)**0.5,
                    reverse=True
                   )
    return dict([(niNnp[0], string.ascii_uppercase[i]) for i, niNnp in enumerate(niNnpL)])    

def entropy2iqrcat(pdO):
    SS = pd.Series(
            [v for c in pdO for v in pdO[c] if not pd.isna(v)] if type(pdO) is pd.DataFrame \
                    else [v for v in pdO if not pd.isna(v)]
        )
    q1, q2, q3 = SS.quantile([.25, .5, .75])
    qL = [-math.inf, q1-3*(q3-q1), q1-1.5*(q3-q1), q1,q2,q3, q3+1.5*(q3-q1), q1+3*(q3-q1), math.inf]
    bL = ['out2n', 'out1n', 'qrt1', 'qrt2', 'qrt3', 'qrt4', 'out1p', 'out2p']
    SS2 = SS.apply(lambda v: bL[[(qL[i] <= v < qL[i+1]) for i in range(8)].index(True)])
    pdO2 = pdO.copy()
    if type(pdO) is pd.DataFrame:
        cNrL = [(c,r) for c in pdO for r in pdO.index if not pd.isna(pdO[c][r])]
        for i, (c, r) in enumerate(cNrL): pdO2.loc[r, c] = SS2[i]
    else:
        iL = [i for i in pdO.index if not pd.isna(pdO[i])]
        for i, i2 in enumerate(iL): pdO2[i2] = SS2[i]
    return pdO2

In [ ]:
pairsDF = get_2v_MIs_for_DF(eDFssa)-get_2v_MIs_for_DF(eDFssa, eDFm)
print(pairsDF)
pairsDF2 = entropy2iqrcat(pairsDF)
print(pairsDF2)
pairsSS = pd.Series([v for c in pairsDF for v in pairsDF[c] if not pd.isna(v)])
pairsSS2 = pd.Series([v for c in pairsDF2 for v in pairsDF2[c] if not pd.isna(v)])
plt.hist(pairsSS, 20)
#print(pairsSS2)

#tripletsSS = get_3v_MIs_for_DF(eDFssa)
#print(tripletsSS)
#tripletsSS2 = entropy2iqrcat(tripletsSS)
#print(tripletsSS2)

#plt.hist(tripletsSS, 42)

In [ ]:
print(pairsSS[pairsSS2=='out2n'])
print(pairsSS[pairsSS2=='out1n'])
plt.hist(pairsSS[pairsSS2=='out2n'], 1)
plt.hist(pairsSS[pairsSS2=='out1n'], 1)
plt.hist(pairsSS[pairsSS2=='qrt1'], 2)
plt.hist(pairsSS[pairsSS2=='qrt2'], 2)
plt.hist(pairsSS[pairsSS2=='qrt3'], 2)
plt.hist(pairsSS[pairsSS2=='qrt4'], 2)
plt.hist(pairsSS[pairsSS2=='out1p'], 2)
plt.hist(pairsSS[pairsSS2=='out2p'], 2)

# plt.hist(tripletsSS[tripletsSS2=='n_out'], 12)
# plt.hist(tripletsSS[tripletsSS2=='q1'], 6)
# plt.hist(tripletsSS[tripletsSS2=='q2'], 3)
# plt.hist(tripletsSS[tripletsSS2=='q3'], 3)
# plt.hist(tripletsSS[tripletsSS2=='q4'], 6)
# plt.hist(tripletsSS[tripletsSS2=='p_out'], 12)

### Functions for drawing elements of graph

In [ ]:
# DRAW TITLE OF GRAPH
def draw_title(Graph, axes, node_positions, title_text, font_color='black', font_size=20, font_family='sans-serif', 
               cmap=None, vmin=None, vmax=None):
    cp = get_graph_center_position(Graph, node_positions)
    max_r = get_max_radius(Graph, node_positions, cp)
    title_xy = cp[0]-max_r*CGRAPH_LABEL_RADIUS_MULTIPLIER, cp[1]+max_r*CGRAPH_LABEL_RADIUS_MULTIPLIER
    axes.annotate(title_text, xy=title_xy, fontsize=font_size, va='top', ha='left', family=font_family, color=font_color)
    return

# DRAW LEGEND OF GRAPH
def draw_legend(Graph, axes, node_positions, 
                size, color='lightgray', 
                cmap=None, vmin=None, vmax=None, alpha=0.7,
                size2=None, color2='gray', 
                cmap2=None, vmin2=None, vmax2=None, alpha2=0.7,  
                size3=None, color3='darkgray', 
                cmap3=None, vmin3=None, vmax3=None, alpha3=0.7,  
                label='', font_size=12, font_family='sans-serif', font_color=FONT_COLOR):
    cp = get_graph_center_position(Graph, node_positions)
    max_r = get_max_radius(Graph, node_positions, cp)
    lgnd_r = max_r*CGRAPH_LEGEND_RADIUS_MULTIPLIER + (max([size, size2, size3]) if size2 else size)     
    lgnd_xy = (cp[0]-lgnd_r*math.cos(math.pi/4), cp[1]+lgnd_r*math.sin(math.pi/4))
    
    if not size2:        
        P1 = Circle(lgnd_xy, radius=size, color=get_color(color, cmap, vmin, vmax), alpha=alpha)
        axes.add_patch(P1)
    else:
        P1 = Wedge(lgnd_xy, r=size, 
           theta1=SPLIT_CIRCLE_ANGLE_0, theta2=SPLIT_CIRCLE_ANGLE_1,
           color = get_color(color,cmap,vmin,vmax), alpha=alpha)
        axes.add_patch(P1)
        P2 = Wedge(lgnd_xy, r=size2, 
           theta1=SPLIT_CIRCLE_ANGLE_1, theta2=SPLIT_CIRCLE_ANGLE_2,
           color = get_color(color2,cmap2,vmin2,vmax2), alpha=alpha2)
        axes.add_patch(P2)
        P3 = Wedge(lgnd_xy, r=size3, 
           theta1=SPLIT_CIRCLE_ANGLE_2, theta2=SPLIT_CIRCLE_ANGLE_0,
           color = get_color(color3,cmap3,vmin3,vmax3), alpha=alpha3)
        axes.add_patch(P3)

    lbl_xy = (cp[0]-lgnd_r*math.cos(math.pi/4), cp[1]+lgnd_r*math.sin(math.pi/4))
    axes.annotate(label, xy=lbl_xy, fontsize=font_size, va='center', ha='center', 
                  family=font_family, color=font_color)    
    return

# DRAW NODES FOR VALUES FOR SINGLETONS
def draw_nodes(Graph, axes, node_positions, 
               sizes=0.01, colors='blue',
               cmap=None, vmin=None, vmax=None, alpha=0.7,
               sizes2=None, colors2=None,
               cmap2=None, vmin2=None, vmax2=None, alpha2=0.7,
               sizes3=None, colors3=None,
               cmap3=None, vmin3=None, vmax3=None, alpha3=0.7,
               with_labels=False, node_labels=None,
               font_size=12, font_family='monospace', #'sans-serif', 
               font_color='black'):
    nodes = Graph.nodes()    
    sizes = sizes if type(sizes) in (list,pd.Series) else pd.Series([sizes]*len(nodes)) 
    colors = colors if type(colors) in (list,pd.Series) else pd.Series([colors]*len(nodes))
    
    if (sizes2 is not None) and not (type(sizes2) in (list,pd.Series) and len(sizes2) == 0):
        sizes2 = sizes2 if type(sizes2) in (list,pd.Series) else pd.Series([sizes2]*len(nodes))
        colors2 = colors2 if type(colors2) in (list,pd.Series) else pd.Series([colors2]*len(nodes))
        sizes3 = sizes3 if type(sizes3) in (list,pd.Series) else pd.Series([sizes3]*len(nodes))
        colors3 = colors3 if type(colors3) in (list,pd.Series) else pd.Series([colors3]*len(nodes))
    else: 
        sizes2 = pd.Series([None]*len(nodes))
        colors2 = pd.Series([None]*len(nodes))
        sizes3 = pd.Series([None]*len(nodes))
        colors3 = pd.Series([None]*len(nodes))
    
    nNsNcNs2Nc2Ns3Nc3L = sorted(list(zip(nodes, sizes, colors, sizes2, colors2, sizes3, colors3)), 
            key = lambda T: (T[1], T[3], T[5]), reverse=False)
    
    for ni, s, c, s2, c2, s3, c3 in nNsNcNs2Nc2Ns3Nc3L:
        
        if not (s2 and c2):
            P1 = Circle(node_positions[ni], radius=s, 
                        color = get_color(c, cmap, vmin, vmax), alpha=alpha)
            axes.add_patch(P1)            
        else:
            P1 = Wedge(node_positions[ni], r=s, 
                       theta1=SPLIT_CIRCLE_ANGLE_0, theta2=SPLIT_CIRCLE_ANGLE_1,
                       color = get_color(c, cmap, vmin, vmax), alpha=alpha)
            axes.add_patch(P1)
            P2 = Wedge(node_positions[ni], r=s2, 
                       theta1=SPLIT_CIRCLE_ANGLE_1, theta2=SPLIT_CIRCLE_ANGLE_2,
                       color = get_color(c2, cmap2, vmin2, vmax2), alpha=alpha2)
            axes.add_patch(P2)
            P3 = Wedge(node_positions[ni], r=s3, 
                       theta1=SPLIT_CIRCLE_ANGLE_2, theta2=SPLIT_CIRCLE_ANGLE_0,
                       color = get_color(c3, cmap3, vmin3, vmax3), alpha=alpha3)
            axes.add_patch(P3)

        if with_labels:   
            l, l2 = node_labels[ni][0], node_labels[ni][1:]
            axes.annotate(l + ' '*len(l2), xy=node_positions[ni], fontsize=font_size, 
                          va='center', ha='center', family=font_family,
                          color = LETTER_COLOR_DICT[l])
            axes.annotate(' ' + l2, xy=node_positions[ni], fontsize=font_size, 
                          va='center', ha='center', family=font_family, 
                          color = font_color)            

    return


# DRAW SLIGHTLY CURVED EDGES FOR VALUES BETWEEN PAIRS

def get_edge_offset(node1_position, node2_position, graph_center_position):
    n1x, n1y, n2x, n2y, gcx, gcy = *node1_position, *node2_position, *graph_center_position
    length = ((n2x-n1x)**2+(n2y-n1y)**2)**0.5
    displace_n1 = ((n1x-gcx)**2+(n1y-gcy)**2)**0.5
    displace_n2 = ((n2x-gcx)**2+(n2y-gcy)**2)**0.5
    acos1 = math.acos((displace_n1**2+displace_n2**2-length**2)/(2*displace_n1*displace_n2))
    acos2 = math.acos((displace_n1**2+length**2-displace_n2**2)/(2*displace_n1*length))
    incidence = math.pi-0.5*acos1-acos2
    return (0.5*math.pi-abs(0.5*math.pi-incidence))/(0.5*math.pi) * EDGE_CURVE_OFFSET_RATIO*length

def get_edge_offset_position(node1_position, node2_position, graph_center_position):
    n1x, n1y, n2x, n2y, gcx, gcy = *node1_position, *node2_position, *graph_center_position
    offset = get_edge_offset(node1_position, node2_position, graph_center_position)
    cx, cy = ((n1x+n2x)/2.0, (n1y+n2y)/2.0)    
    edge_slope = (n2y-n1y)/(n2x-n1x) if (n2x-n1x) != 0.0 else 1000000.0
    orth_angle = math.atan(-1/edge_slope if edge_slope != 0 else 1000000.0)
    x_offset, y_offset = offset*math.cos(orth_angle), offset*math.sin(orth_angle)
    ox, oy = None, None
    if edge_slope >= 0:
        if cx >= gcx and cy <= gcy: ox, oy = cx+x_offset, cy+y_offset
        elif cx <= gcx and cy >= gcy: ox, oy = cx-x_offset, cy-y_offset
        elif cx >= gcx and cy >= gcy:
            if edge_slope >= 1: ox, oy = cx+x_offset, cy+y_offset
            else: ox, oy = cx-x_offset, cy-y_offset
        else:
            if edge_slope <= 1: ox, oy = cx+x_offset, cy+y_offset
            else: ox, oy = cx-x_offset, cy-y_offset       
    else:
        if cx >= gcx and cy >= gcy: ox, oy = cx+x_offset, cy+y_offset
        elif cx <= gcx and cy <= gcy: ox, oy = cx-x_offset, cy-y_offset
        elif cx <= gcx and cy >= gcy:
            if edge_slope >= -1: ox, oy = cx+x_offset, cy+y_offset
            else: ox, oy = cx-x_offset, cy-y_offset
        else:
            if edge_slope <= -1: ox, oy = cx+x_offset, cy+y_offset
            else: ox, oy = cx-x_offset, cy-y_offset
    return (ox, oy)

    draw_edges(Graph, ax, node_positions, edge_colors=edge_colors, edge_width=3, 
               cmap=color_map, vmin=color_min, vmax=color_max)

def draw_edges(Graph, axes, node_positions, 
               colors='blue', width=2, style='-', cmap=None, vmin=None, vmax=None, alpha=1.0,
               colors2=None, width2=None, style2='-', cmap2=None, vmin2=None, vmax2=None, alpha2=1.0,
               highlights=None, highlight_color=None, highlight_width=2, highlight_style='-'):
    edges = Graph.edges()
    colors = colors if type(colors) in (list,pd.Series) else [colors]*len(edges)

    if (colors2 is not None) and not (type(colors2) in (list,pd.Series) and len(colors2)==0):    
        colors2 = colors2 if type(colors2) in (list,pd.Series) else [colors2]*len(edges)
    else:
        colors2 = [None]*len(edges)   
    if (highlights is not None) and not(type(highlights) in (list,pd.Series) and len(highlights)==0):
        highlights = highlights if type(highlights) in (list,pd.Series) else [highlights]*len(edges)
    else:
        highlights = [None]*len(edges)
        
    cxy = get_graph_center_position(Graph, node_positions)
    eNcNc2NhL = sorted(list(zip(edges, colors, colors2, highlights)), 
                     key=lambda T: (abs(T[1]), (abs(T[2]) if T[2] else None)))   

    for (n1i, n2i), c, c2, h in eNcNc2NhL:
        if h and (h in HIGHLIGHT_DICT):
            n1xy, n2xy = tuple(node_positions[n1i]), tuple(node_positions[n2i])
            oxy = get_edge_offset_position(n1xy, n2xy, cxy)
            Path = mpath.Path
            P = PathPatch(Path([n1xy, oxy, n2xy], [Path.MOVETO, Path.CURVE3, Path.CURVE3]),
                           color=highlight_color if highlight_color else HIGHLIGHT_DICT[h], 
                           linewidth=highlight_width, linestyle=highlight_style,
                           transform=axes.transData, fc='None', alpha=1.0
                        )
            axes.add_patch(P)        

    for (n1i, n2i), c, c2, h in eNcNc2NhL:
        n1xy, n2xy = tuple(node_positions[n1i]), tuple(node_positions[n2i])
        oxy = get_edge_offset_position(n1xy, n2xy, cxy)
        Path = mpath.Path
        if not c2:
            P1 = PathPatch(Path([n1xy, oxy, n2xy], [Path.MOVETO, Path.CURVE3, Path.CURVE3]),
                           color = get_color(c, cmap, vmin, vmax), 
                           linewidth=width, linestyle=style, 
                           transform=axes.transData, fc='None', alpha=alpha
                        )
            axes.add_patch(P1)
        else:
            clr1, clr2, w1, a1, w2, a2 = None, None, None, None, None, None
            if width > width2:
                clr1, clr2 = get_color(c, cmap, vmin, vmax), get_color(c2, cmap2, vmin2, vmax2)
                w1, a1, w2, a2 = width, alpha, width2, alpha2
            else:
                clr1, clr2 = get_color(c2, cmap2, vmin2, vmax2), get_color(c, cmap, vmin, vmax)
                w1, a1, w2, a2 = width2, alpha2, width, alpha
            P1 = PathPatch(Path([n1xy, oxy, n2xy], [Path.MOVETO, Path.CURVE3, Path.CURVE3]),
                              color = clr1, linewidth=w1, linestyle=style, 
                              transform=axes.transData, fc='None', alpha=a1
                            )
            axes.add_patch(P1)
            P2 = PathPatch(Path([n1xy, oxy, n2xy], [Path.MOVETO, Path.CURVE3, Path.CURVE3]),
                              color = clr2, linewidth=w2, linestyle=style2, 
                              transform=axes.transData, fc='None', alpha=a2
                            )
            axes.add_patch(P2)
            
    return


# DRAW LABELED OUTCIRCLE AND REFERENCE LINES

def get_outcircle_angles(node_positions):
    nc = len(node_positions)
    ec = int(math.factorial(nc)/(math.factorial(3)*math.factorial(nc-3)))
    return [i*(2*math.pi)/ec for i in range(ec)]
        
def get_outcircle_positions(center_position, max_radius, outcircle_angles):
    radius = max_radius * CGRAPH_RADIUS_MULTIPLIER
    return [(center_position[0]+radius*math.cos(a), center_position[1]+radius*math.sin(a)) for a in outcircle_angles]

def get_outcircle_label_positions(center_position, max_radius, outcircle_angles):
    radius = max_radius * CGRAPH_LABEL_RADIUS_MULTIPLIER
    return [(center_position[0]+radius*math.cos(a), center_position[1]+radius*math.sin(a)) for a in outcircle_angles]
                
def get_outcircle_triplet_coordinate_info(Graph, node_positions, node_triplets):
    center_position = get_graph_center_position(Graph, node_positions)
    apL, dpL, dhL, aL = [], [], [], []
    for nitT in node_triplets:
        np1, np2, np3 = node_positions[nitT[0]], node_positions[nitT[1]], node_positions[nitT[2]]
        mx, my = ((np1[0]+np2[0]+np3[0])/3.0, (np1[1]+np2[1]+np3[1])/3.0) 
        dmx, dmy = mx-center_position[0], my-center_position[1]
        dmh = (dmx**2+dmy**2)**0.5
        acos = math.acos(dmx/dmh)
        a = acos if dmy >= 0 else 2*math.pi-acos
        apL.append((mx, my))
        dpL.append((dmx, dmy))
        dhL.append(dmh)
        aL.append(a)
    return pd.DataFrame({'abs_position': apL, 'diff_position': dpL, 'diff_hypotenuse': dhL, 'angle': aL},  index = node_triplets)

def assign_outcircle_positions(Graph, node_positions, node_sizes, triplet_coordinate_info):

    center_position = get_graph_center_position(Graph, node_positions)
    max_radius = get_max_radius(Graph, node_positions, center_position)
    outcircle_angles = get_outcircle_angles(node_positions) 
    outcircle_positions = get_outcircle_positions(center_position, max_radius, outcircle_angles)
    outcircle_label_positions = get_outcircle_label_positions(center_position, max_radius, outcircle_angles)
    
    node_triplets = get_node_triplets(Graph)
    tNsL = sorted(zip(node_triplets, node_sizes), key=lambda tNs: tNs[1], reverse=True)
    
    nodes, outcircle_count = list(Graph.nodes()), len(outcircle_angles)
    tripletL, tposL = [None]*outcircle_count, [None]*outcircle_count
    for nitT in [tNs[0] for tNs in tNsL]:
        a, tpos = triplet_coordinate_info['angle'][nitT], triplet_coordinate_info['abs_position'][nitT]
        si = int(round((a/(2*math.pi))*outcircle_count))
        if si == outcircle_count: si = 0
        for ci in range(outcircle_count):
            di, dd = int((ci+1)//2), bool((ci+1)%2)
            i = si+di if dd else si-di
            if i < 0: i = outcircle_count-(-i)
            elif i >= outcircle_count: i = i-outcircle_count
            if not tripletL[i]:
                tripletL[i], tposL[i] = nitT, tpos
                break
                
    return pd.DataFrame({'angle': outcircle_angles, 'position': outcircle_positions, 'triplet_position': tposL, 
                         'label_position': outcircle_label_positions}, index=tripletL)
    
def draw_outcircle(Graph, axes, node_positions, 
                   sizes=0.005, colors='blue', width=0.5, style='-',
                   cmap=None, vmin=None, vmax=None, alpha=1.0,
                   sizes2=None, colors2=None, 
                   cmap2=None, vmin2=None, vmax2=None, alpha2=1.0,
                   sizes3=None, colors3=None, 
                   cmap3=None, vmin3=None, vmax3=None, alpha3=1.0,
                   highlights=None, highlight_color=None, highlight_width=0.5, highlight_style='-',
                   node_letters=None,  
                   font_color='darkgray', font_size=10, font_family='monospace'):

    triplets = get_node_triplets(Graph)
    sizes = sizes if type(sizes) in (list,pd.Series) else pd.Series([sizes]*len(triplets))
    colors = colors if type(colors) in (list,pd.Series) else pd.Series([colors]*len(triplets))
    if (highlights is not None) and not(type(highlights) in (list,pd.Series) and len(highlights)==0):
        highlights=highlights if type(highlights) in (list,pd.Series) else [highlights]*len(triplets)
    else:
        highlights = [None]*len(triplets)
    
    if sizes2 is not None and colors2 is not None:
        sizes2 = sizes2 if type(sizes2) in (list,pd.Series) else pd.Series([sizes2]*len(triplets))
        colors2=colors2 if type(colors2) in (list,pd.Series) else pd.Series([colors2]*len(triplets))
        sizes3 = sizes3 if type(sizes3) in (list,pd.Series) else pd.Series([sizes3]*len(triplets))
        colors3=colors3 if type(colors3) in (list,pd.Series) else pd.Series([colors3]*len(triplets))

    else: 
        sizes2 = pd.Series([None]*len(triplets))
        colors2 = pd.Series([None]*len(triplets))
        sizes3 = pd.Series([None]*len(triplets))
        colors3 = pd.Series([None]*len(triplets))
        
    tripletDF = get_outcircle_triplet_coordinate_info(Graph, node_positions, triplets)
    assignedDF = assign_outcircle_positions(Graph, node_positions, sizes, tripletDF)
    tNsNcNs2Nc2Ns3Nc3NhL = sorted(
            list(zip(triplets, sizes, colors, sizes2, colors2, sizes3, colors3, highlights)),
                    key = lambda T: (T[1], T[3], T[5]), reverse=False
        )
    
    for niT, s, c, s2, c2, s3, c3, h in tNsNcNs2Nc2Ns3Nc3NhL:
        if h and (h in HIGHLIGHT_DICT): 
            oc_xy, t_xy = assignedDF['position'][niT], assignedDF['triplet_position'][niT]
            for n_xy in [node_positions[ni] for ni in niT]:
                Path = mpath.Path            
                P = PathPatch(Path([n_xy, oc_xy], [Path.MOVETO, Path.LINETO]),
                               color=highlight_color if highlight_color else HIGHLIGHT_DICT[h],
                               linewidth=highlight_width, linestyle=highlight_style,
                               transform=axes.transData, fc='None', alpha=1.0
                            )
                axes.add_patch(P)    
    for niT, s, c, s2, c2, s3, c3, h in tNsNcNs2Nc2Ns3Nc3NhL:
        oc_xy, t_xy = assignedDF['position'][niT], assignedDF['triplet_position'][niT]
        for n_xy in [node_positions[ni] for ni in niT]:
            Path = mpath.Path            
            P1 = PathPatch(Path([n_xy, oc_xy], [Path.MOVETO, Path.LINETO]),
                           color = get_color(c, cmap, vmin, vmax), 
                           linewidth=width, linestyle=style,
                           transform=axes.transData, fc='None', alpha=alpha
                        )
            axes.add_patch(P1)
        if not (s2 and c2):
            P3 = Circle(oc_xy, radius=s, color = get_color(c, cmap, vmin, vmax), alpha=alpha)
            axes.add_patch(P3)            
        else:
            P3 = Wedge(oc_xy, r=s, 
                       theta1=SPLIT_CIRCLE_ANGLE_0, theta2=SPLIT_CIRCLE_ANGLE_1,
                       color = get_color(c,cmap,vmin,vmax), alpha=alpha)
            axes.add_patch(P3)
            P4 = Wedge(oc_xy, r=s2, 
                       theta1=SPLIT_CIRCLE_ANGLE_1, theta2=SPLIT_CIRCLE_ANGLE_2,
                       color = get_color(c2,cmap2,vmin2,vmax2), alpha=alpha2)
            axes.add_patch(P4)
            P5 = Wedge(oc_xy, r=s3, 
                       theta1=SPLIT_CIRCLE_ANGLE_2, theta2=SPLIT_CIRCLE_ANGLE_0,
                       color = get_color(c3,cmap3,vmin3,vmax3), alpha=alpha3)
            axes.add_patch(P5)
        if node_letters:
            lbl_xy, lbl_a = assignedDF['label_position'][niT], assignedDF['angle'][niT]
            if math.pi/2.0 <lbl_a < 3*math.pi/2.0: lbl_a += math.pi
            lL = sorted([node_letters[ni] for ni in niT])
            for i in range(3):
                l = lL[i]
                lbl, tc = ' '*i + l + ' '*(2-i), LETTER_COLOR_DICT[l]
                axes.annotate(lbl, xy=lbl_xy, fontsize=font_size, ha='center', 
                              family=font_family, color=tc,
                              rotation=math.degrees(lbl_a)) 
    return

In [ ]:
#plt.cm.binary(np.linspace(0., 1, 128))
#plt.get_cmap(ENTROPY_CMAP

### Function to render a graph representing the entropy distribution of the table...

In [ ]:
def render_entropy_graph(DF, cDF=None, target_field=None, outfile=None):
    
    # GET ENTROPY, CROSS ENTROPY(?), MUTUAL INFORMATION TO BE VISUALIZED
    
    legendValue, legendValue2 = None, None
    scale, title, label = None, None, None
    singletonsSS, steppedSingletonsDF, steppedSingletonsDF_color = None, None, None
    pairsDF, tripletsSS = None, None
    singletons2SS, steppedSingletons2DF, steppedSingletons2DF_color = None, None, None 
    pairs2DF, triplets2SS = None, None
    Graph, nodePositions = None, None
    if not target_field:
        
        legendValue = te = get_jE_for_DF(DF)
        scale = get_scale(te, RADIUS_FOR_TOTAL_ENTROPY)
        title, label = GRAPH_TITLE, GRAPH_DATASET_LABEL % (te)
        singletonsSS = get_Es_for_DF(DF)
        sceDF = get_stepped_cE_data_for_DF(DF)
        imiDF, ceiDF = sceDF.applymap(lambda T: T[1]), sceDF.applymap(lambda T: T[2])
        steppedSingletonsDF, steppedSingletonsDF_color = ceiDF, -imiDF
        pairsDF = -get_2v_MIs_for_DF(DF)
        tripletsSS = -get_3v_MIs_for_DF(DF)

        if cDF is not None:
            
            legendValue2 = te2 = get_jE_for_DF(DF, cDF=cDF)
            scale = get_scale(max([te, te2]), RADIUS_FOR_TOTAL_ENTROPY)
            singletons2SS = get_Es_for_DF(DF, cDF=cDF)
            sce2DF = get_stepped_cE_data_for_DF(DF, cDF=cDF)
            imi2DF, cei2DF = sce2DF.applymap(lambda T: T[1]), sce2DF.applymap(lambda T: T[2])
            steppedSingletons2DF, steppedSingletons2DF_color = cei2DF, -imi2DF
            pairs2DF = -get_2v_MIs_for_DF(DF, cDF=cDF)
            triplets2SS = -get_3v_MIs_for_DF(DF, cDF=cDF)
            
            legendValue3 = te3 = get_jE_for_DF(cDF)
            scale = get_scale(max([te, te2, te3]), RADIUS_FOR_TOTAL_ENTROPY)
            singletons3SS = get_Es_for_DF(cDF)
            sce3DF = get_stepped_cE_data_for_DF(cDF)
            imi3DF, cei3DF = sce3DF.applymap(lambda T: T[1]), sce3DF.applymap(lambda T: T[2])
            steppedSingletons3DF, steppedSingletons3DF_color = cei3DF, -imi3DF
            pairs3DF = -get_2v_MIs_for_DF(cDF)
            triplets3SS = -get_3v_MIs_for_DF(cDF)
            
        ewDF = get_2v_MIs_for_DF(DF)
        ewDF1 = (ewDF-(ewDF.min().min())).fillna(0)
        G1 = nx.from_numpy_matrix(np.matrix(ewDF1))
        np1 = nx.spring_layout(G1, iterations=10000)
        ewDF2 = (np.absolute(ewDF.fillna(0))**(1))*np.sign(ewDF.fillna(0))
        Graph = nx.from_numpy_matrix(np.matrix(ewDF2))
        nodePositions = nx.spring_layout(Graph, iterations=10000, pos=np1)
                         
    else:
        pass
       
    # TRANSLATE ENTROPY, CROSS ENTROPY(?), MUTUAL INFORMATION INTO ELEMENTS OF THE GRAPH
    
    fields, steps = steppedSingletonsDF.index, steppedSingletonsDF.columns
    legendRadius= entropy2radii(legendValue)*scale
    bgRadiiSS = entropy2radii(singletonsSS)*scale
    nodeRadiiDF = entropy2radii(steppedSingletonsDF)*scale    
    tripletRadiiSS = entropy2radii(tripletsSS)*scale
    bgNodeColorSS = entropy2color(singletonsSS)
    nodeColorDF = entropy2color(steppedSingletonsDF_color)
    edgeColorDF = entropy2color(pairsDF)
    tripletColorSS = entropy2color(tripletsSS)
    nodeLettersD = get_node_letters(Graph, nodePositions)
    nodeLabelsD = dict([(i, "%s: %s" % (nodeLettersD[i], fields[i])) for i in range(len(fields))])
    
    bgNodeColorL = [bgNodeColorSS[fields[i]] for i in Graph.nodes()]
    ncL = [nodeColorDF[c][fields[i]] for c in nodeColorDF.columns for i in Graph.nodes()]    
    edgeColorL = [edgeColorDF[fields[i2]][fields[i1]] for (i1,i2) in Graph.edges()]
    tripletColorL = [tripletColorSS[(fields[i1],fields[i2],fields[i3])] \
                     for (i1,i2,i3) in get_node_triplets(Graph)]
                   
    legendRadius2, bgRadii2SS, nodeRadii2DF, tripletRadii2SS = None, None, None, None
    bgNodeColor2SS, nodeColor2DF, edgeColor2DF, tripletColor2SS = None, None, None, None
    bgNodeColor2L, edgeColor2L, tripletColor2L = [], [], []

    legendRadius3, bgRadii3SS, nodeRadii3DF, tripletRadii3SS = None, None, None, None
    bgNodeColor3SS, nodeColor3DF, edgeColor3DF, tripletColor3SS = None, None, None, None
    bgNodeColor3L, edgeColor3L, tripletColor3L = [], [], []
    edgeHighlightDF, tripletHighlightSS = None, None
    edgeHighlightL, tripletHighlightL = [], []

    if cDF is not None:        
        legendRadius2 = entropy2radii(legendValue2)*scale
        bgRadii2SS = entropy2radii(singletons2SS)*scale
        nodeRadii2DF = entropy2radii(steppedSingletons2DF)*scale    
        tripletRadii2SS = entropy2radii(triplets2SS)*scale
        bgNodeColor2SS = entropy2color(singletons2SS)
        nodeColor2DF = entropy2color(steppedSingletons2DF_color)
        edgeColor2DF = entropy2color(pairs2DF)
        tripletColor2SS = entropy2color(triplets2SS)        
        bgNodeColor2L = [bgNodeColor2SS[fields[i]] for i in Graph.nodes()]
        edgeColor2L = [edgeColor2DF[fields[i2]][fields[i1]] for (i1,i2) in Graph.edges()] 
        tripletColor2L = [tripletColor2SS[(fields[i1],fields[i2],fields[i3])] \
                          for (i1,i2,i3) in get_node_triplets(Graph)]  
        
        legendRadius3 = entropy2radii(legendValue3)*scale
        bgRadii3SS = entropy2radii(singletons3SS)*scale
        nodeRadii3DF = entropy2radii(steppedSingletons3DF)*scale
        tripletRadii3SS = entropy2radii(triplets3SS)*scale
        bgNodeColor3SS = entropy2color(singletons3SS)
        nodeColor3DF = entropy2color(steppedSingletons3DF_color)
        edgeColor3DF = entropy2color(pairs3DF)
        tripletColor3SS = entropy2color(triplets3SS)
        bgNodeColor3L = [bgNodeColor3SS[fields[i]] for i in Graph.nodes()]
        edgeColor3L = [edgeColor3DF[fields[i2]][fields[i1]] for (i1,i2) in Graph.edges()] 
        tripletColor3L = [tripletColor3SS[(fields[i1],fields[i2],fields[i3])] \
                          for (i1,i2,i3) in get_node_triplets(Graph)]  
        
        
        edgeHighlightDF = entropy2iqrcat(pairsDF-pairs2DF)
        tripletHighlightSS = entropy2iqrcat(tripletsSS-triplets2SS)
        edgeHighlightL = [edgeHighlightDF[fields[i2]][fields[i1]] for (i1,i2) in Graph.edges()] 
        tripletHighlightL = [tripletHighlightSS[(fields[i1],fields[i2],fields[i3])] \
                             for (i1,i2,i3) in get_node_triplets(Graph)]
        
    ac2A = np.absolute(ncL + edgeColorL + tripletColorL + bgNodeColorL + \
                       edgeColor2L + tripletColor2L + bgNodeColor2L + \
                       edgeColor3L + tripletColor3L + bgNodeColor3L)
    colorMin, colorMax = -max(ac2A), max(ac2A)
        
    # DRAW GRAPH
        
    fig = plt.figure(num=None, figsize=(GRAPH_WIDTH, GRAPH_HEIGHT), dpi=GRAPH_DOTS_PER_INCH)
    ax = plt.gca()
    ax.set_facecolor(BACKGROUND_COLOR)
    plt.tick_params(axis='both', which='both', bottom=False, top=False, 
                    labelbottom=False, labelleft=False)
    plt.axis('equal')
    #plt.rc('text', usetex=True)
    
    draw_title(Graph, ax, nodePositions, title, font_color=FONT_COLOR)
    
    draw_legend(Graph, ax, nodePositions,                 
                size=legendRadius, color=colorMax*0.9, 
                cmap=ENTROPY_CMAP, vmin=colorMin, vmax=colorMax, alpha=0.9,
                size2=legendRadius2, color2=colorMax, 
                cmap2=ENTROPY_CMAP, vmin2=colorMin, vmax2=colorMax, alpha2=0.9,
                size3=legendRadius3, color3=colorMax*0.9, 
                cmap3=ENTROPY_CMAP, vmin3=colorMin, vmax3=colorMax, alpha3=0.9,
                label=label, font_color=FONT_COLOR)
        
    draw_outcircle(Graph, ax, nodePositions, 
                   sizes=tripletRadiiSS, colors=tripletColorL, width=1,
                   cmap=ENTROPY_CMAP2, vmin=colorMin, vmax=colorMax, alpha=1.0,
                   sizes2=tripletRadii2SS, colors2=tripletColor2L,
                   cmap2=ENTROPY_CMAP2, vmin2=colorMin, vmax2=colorMax, alpha2=1.0,
                   sizes3=tripletRadii3SS, colors3=tripletColor3L,
                   cmap3=ENTROPY_CMAP2, vmin3=colorMin, vmax3=colorMax, alpha3=1.0,
                   highlights=tripletHighlightL, highlight_width=2,
                   node_letters=nodeLettersD, font_color=FONT_COLOR)
    
    draw_nodes(Graph, ax, nodePositions, 
               sizes=bgRadiiSS, colors=bgNodeColorL, 
               cmap=ENTROPY_CMAP, vmin=colorMin, vmax=colorMax, alpha=0.9,
               sizes2=bgRadii2SS, colors2=bgNodeColor2L, 
               cmap2=ENTROPY_CMAP, vmin2=colorMin, vmax2=colorMax, alpha2=0.9,
               sizes3=bgRadii3SS, colors3=bgNodeColor3L, 
               cmap3=ENTROPY_CMAP, vmin3=colorMin, vmax3=colorMax, alpha3=0.9,
               with_labels=False)
    
    draw_edges(Graph, ax, nodePositions, 
               colors=edgeColorL, width=3, 
               cmap=ENTROPY_CMAP2, vmin=colorMin, vmax=colorMax, alpha=1.0,
               colors2=edgeColor2L, width2=2, 
               cmap2=ENTROPY_CMAP2, vmin2=colorMin, vmax2=colorMax, alpha2=1.0,
               highlights=edgeHighlightL, highlight_width=5)
        
    for i, step in enumerate(steps):
        draw_nodes(Graph, ax, nodePositions, 
                   sizes=nodeRadiiDF[step], 
                   colors=nodeColorDF[step], 
                   cmap=ENTROPY_CMAP2, vmin=colorMin, vmax=colorMax, alpha=1.0,
                   sizes2=nodeRadii2DF[step] if nodeRadii2DF is not None else None, 
                   colors2=nodeColor2DF[step] if nodeColor2DF is not None else None, 
                   cmap2=ENTROPY_CMAP2, vmin2=colorMin, vmax2=colorMax, alpha2=1.0, 
                   sizes3=nodeRadii3DF[step] if nodeRadii3DF is not None else None, 
                   colors3=nodeColor3DF[step] if nodeColor3DF is not None else None, 
                   cmap3=ENTROPY_CMAP2, vmin3=colorMin, vmax3=colorMax, alpha3=1.0, 
                   with_labels=(i+1==len(steps)), 
                   node_labels=nodeLabelsD, font_color=FONT_COLOR
                )
    
    ax.autoscale()
    if outfile: 
        plt.savefig(outfile, bbox_inches='tight')
    plt.show()
    plt.close(fig)
    
    return

### Execute the function for the given data set...

In [ ]:
render_entropy_graph(eDF, outfile='test.png')

In [ ]:
render_entropy_graph(eDF, cDF=eDF2, outfile='test.png')

In [ ]:
dLL = (get_2v_MIs_for_DF(eDFssa)-get_2v_MIs_for_DF(eDFssa, eDFm)).values.tolist()
dL = [v for v in itertools.chain.from_iterable(dLL) if not pd.isna(v)]
print(stats.skewtest(dL)) 
print(stats.kurtosistest(dL))
plt.hist(dL, bins=20)

In [ ]:
dL = list(get_3v_MIs_for_DF(eDFssa)-get_3v_MIs_for_DF(eDFssa, cDF=eDFm))
print(stats.skewtest(dL)) 
print(stats.kurtosistest(dL))
plt.hist(dL, bins=20)

In [ ]:
dLL = (get_2v_MIs_for_DF(eDFssa)-get_2v_MIs_for_DF(eDFm)).values.tolist()
dL = [v for v in itertools.chain.from_iterable(dLL) if not pd.isna(v)]
print(stats.skewtest(dL)) 
print(stats.kurtosistest(dL))
plt.hist(dL, bins=20)

In [ ]:
dL = list(get_3v_MIs_for_DF(eDFssa)-get_3v_MIs_for_DF(eDFm))
print(stats.skewtest(dL)) 
print(stats.kurtosistest(dL))
plt.hist(dL, bins=20)

### Test visualization for drilling down into target field

In [ ]:
TARGET_FIELD = 'fog'
render_entropy_graph(eDFssa, target_field=TARGET_FIELD, outfile='test2.png')

In [ ]:
TARGET_FIELD = 'fog'
render_entropy_graph(eDFssa, target_field=TARGET_FIELD, cDF=eDFm, outfile='test2.png')

In [ ]:
dLL = (get_3v_MIs_for_DF_and_target_field(eDFssa, TARGET_FIELD) \
       - get_3v_MIs_for_DF_and_target_field(eDFssa, TARGET_FIELD, cDF=eDFm)).values.tolist()
dL = [v for v in itertools.chain.from_iterable(dLL) if not math.isnan(v)]
print(stats.skewtest(dL)) 
print(stats.kurtosistest(dL))
plt.hist(dL, bins=20)

In [ ]:
dL = list(get_4v_MIs_for_DF_and_target_field(eDFssa, target_field=TARGET_FIELD) \
          - get_4v_MIs_for_DF_and_target_field(eDFssa, target_field=TARGET_FIELD, cDF=eDFm))
print(stats.skewtest(dL)) 
print(stats.kurtosistest(dL))
plt.hist(dL, bins=20)

## OLD CODE...

In [ ]:
        legendValue = te = get_E_for_SS(DF[target_field])
        scale = get_scale(te, RADIUS_FOR_TOTAL_MUTUAL_INFORMATION)
        title, label = GRAPH_TITLE_2 % (target_field), GRAPH_DATASET_LABEL_2 % (target_field, te)
        singletonsSS = get_2v_MIs_for_DF_and_target_field(DF, target_field)
        scmiDF = get_stepped_cMI_data_for_DF_and_target_field(DF, target_field)
        imiDF, cmiDF = scmiDF.applymap(lambda T: T[1]), scmiDF.applymap(lambda T: T[2])
        steppedSingletonsDF, steppedSingletonsDF_color = cmiDF, -imiDF
        pairsDF = -get_3v_MIs_for_DF_and_target_field(DF, target_field)
        tripletsSS = -get_4v_MIs_for_DF_and_target_field(DF, target_field)
        
        if cDF is not None:
            legendValue2 = te2 = get_E_for_SS(DF[target_field],cSS=cDF[target_field])
            scale = get_scale(max([te, te2]), RADIUS_FOR_TOTAL_MUTUAL_INFORMATION)
            singletons2SS = get_2v_MIs_for_DF_and_target_field(DF, target_field, cDF=cDF)
            scmi2DF = get_stepped_cMI_data_for_DF_and_target_field(DF, target_field, cDF=cDF)
            imi2DF, cmi2DF = scmi2DF.applymap(lambda T: T[1]), scmi2DF.applymap(lambda T: T[2])
            steppedSingletons2DF, steppedSingletons2DF_color = cmi2DF, -imi2DF
            pairs2DF = -get_3v_MIs_for_DF_and_target_field(DF, target_field, cDF=cDF)
            triplets2SS = -get_4v_MIs_for_DF_and_target_field(DF, target_field, cDF=cDF)
            
        ewDF = get_3v_2WC_info_for_DF_and_target_field(DF, target_field)
        ewDF1 = (ewDF-(ewDF.min().min())).fillna(0)
        G1 = nx.from_numpy_matrix(np.matrix(ewDF1))
        np1 = nx.spring_layout(G1, iterations=10000)
        ewDF2 = (np.absolute(ewDF.fillna(0))**(2))*np.sign(ewDF.fillna(0))
        Graph = nx.from_numpy_matrix(np.matrix(ewDF2))
        nodePositions = nx.spring_layout(Graph, iterations=10000, pos=np1)

        
# def test(tDF, cDF):
#     tDFrs, cDFrs = _synthesize_by_field_DF(tDF), _synthesize_by_field_DF(cDF)
#     tDFjs, cDFjs = _join_split_DFs(tDF, cDF)    

#     miSStrs, miSScrs, miSSxrs, miSStjs, miSScjs, miSSxjs = None, None, None, None, None, None
#     if True:
#         miDFt, miDFc = get_2v_MIs_for_DF(tDF), get_2v_MIs_for_DF(cDF)
#         miDFx = get_2v_MIs_for_DF(tDF, cDF)
#         miDFtrs, miDFcrs = get_2v_MIs_for_DF(tDFrs), get_2v_MIs_for_DF(cDFrs)
#         miDFxrs = get_2v_MIs_for_DF(tDFrs, cDFrs)
#         miDFtjs, miDFcjs = get_2v_MIs_for_DF(tDFjs), get_2v_MIs_for_DF(cDFjs)
#         miDFxjs = get_2v_MIs_for_DF(tDFjs, cDFjs)
#         icL = [(i, c) for c in miDFt for i in miDFt.index if not pd.isna(miDFt[c][i])]
#         miSSt = pd.Series([v for c in miDFt for v in miDFt[c] if not pd.isna(v)], index=icL)
#         miSSc = pd.Series([v for c in miDFc for v in miDFc[c] if not pd.isna(v)], index=icL)
#         miSSx = pd.Series([v for c in miDFx for v in miDFx[c] if not pd.isna(v)], index=icL)
#         miSStrs = pd.Series([v for c in miDFtrs for v in miDFtrs[c] if not pd.isna(v)], index=icL)
#         miSScrs = pd.Series([v for c in miDFcrs for v in miDFcrs[c] if not pd.isna(v)], index=icL)
#         miSSxrs = pd.Series([v for c in miDFxrs for v in miDFxrs[c] if not pd.isna(v)], index=icL)
#         miSStjs = pd.Series([v for c in miDFtjs for v in miDFtjs[c] if not pd.isna(v)], index=icL)
#         miSScjs = pd.Series([v for c in miDFcjs for v in miDFcjs[c] if not pd.isna(v)], index=icL)
#         miSSxjs = pd.Series([v for c in miDFxjs for v in miDFxjs[c] if not pd.isna(v)], index=icL)
#     else:
#         miSSt, miSSc = get_3v_MIs_for_DF(tDF), get_3v_MIs_for_DF(cDF)
#         miSSx = get_3v_MIs_for_DF(tDF, cDF)
#         miSStrs, miSScrs = get_3v_MIs_for_DF(tDFrs), get_3v_MIs_for_DF(cDFrs)
#         miSSxrs = get_3v_MIs_for_DF(tDFrs, cDFrs)
#         miSStjs, miSScjs = get_3v_MIs_for_DF(tDFjs), get_3v_MIs_for_DF(cDFjs)
#         miSSxjs = get_3v_MIs_for_DF(tDFjs, cDFjs)
        
        
#     with plt.rc_context({'xtick.color': 'white', 'ytick.color': 'white', 'text.color': 'white'}):
#         plt.figure(1, figsize=(12, 18))

        
#         plt.subplot(3, 2, 1)
#         plt.title('Non-normalized Test vs Control')        
#         plt.plot(miSStjs, 0.0+1.0*miSStjs, '-', color='green', alpha=0.5)
#         plt.scatter(miSSt, miSSc, marker='.', alpha=0.5, 
#                     c=pd.Series(range(miSSt.size))/miSSt.size, cmap=plt.get_cmap('tab20'))
#         plt.scatter(miSStjs, miSScjs, marker='.', alpha=0.1, color='gray')
#         plt.scatter(miSStrs, miSScrs, marker='.', alpha=1.0, color='black')

        
#         plt.subplot(3, 2, 2)
#         plt.title('Non-normalized Test vs Cross')
#         plt.plot(miSStjs, 0.0+1.0*miSStjs, '-', color='green', alpha=0.5)
#         plt.scatter(miSSt, miSSx, marker='.', alpha=0.5, 
#                     c=pd.Series(range(miSSt.size))/miSSt.size, cmap=plt.get_cmap('tab20'))
#         plt.scatter(miSStjs, miSSxjs, marker='.', alpha=0.1, color='gray')
#         plt.scatter(miSStrs, miSSxrs, marker='.', alpha=1.0, color='black')

        
#         plt.subplot(3, 2, 3)
#         plt.title('Non-normalized Test vs Control (filtered)')
                        
#         b1rs, m1rs, fL1rs, oL1rs = get_filtered_linear_least_squares_fit(miSStrs, miSScrs)
#         b1js, m1js, fL1js, oL1js = get_filtered_linear_least_squares_fit(miSStjs, miSScjs)

#         plt.plot(miSStjs, b1rs+m1rs*miSStjs, '-', color='blue', alpha=0.5)
#         plt.plot(miSStjs, b1js+m1js*miSStjs, '-', color='red', alpha=0.5)

#         plt.scatter(miSSt, miSSc, marker='.', alpha=0.5, 
#                     c=pd.Series(range(miSSt.size))/miSSt.size, cmap=plt.get_cmap('tab20'))

#         plt.scatter(miSStrs[fL1rs], miSScrs[fL1rs], marker='+', alpha=1.0, color='black')
#         plt.scatter(miSStrs[oL1rs], miSScrs[oL1rs], marker='+', alpha=1.0, color='blue')

#         plt.scatter(miSStjs[fL1js], miSScjs[fL1js], marker='.', alpha=1.0, color='black')
#         plt.scatter(miSStjs[oL1js], miSScjs[oL1js], marker='.', alpha=1.0, color='red')
        
        
#         plt.subplot(3, 2, 4)
#         plt.title('Non-normalized Test vs Cross (filtered)')        

#         b2rs, m2rs, fL2rs, oL2rs = get_filtered_linear_least_squares_fit(miSStrs, miSSxrs)
#         b2js, m2js, fL2js, oL2js = get_filtered_linear_least_squares_fit(miSStjs, miSSxjs)

#         plt.plot(miSStjs, b2rs+m2rs*miSStjs, '-', color='blue', alpha=0.5)
#         plt.plot(miSStjs, b2js+m2js*miSStjs, '-', color='red', alpha=0.5)

#         plt.scatter(miSSt, miSSx, marker='.', alpha=0.5,
#                     c=pd.Series(range(miSSt.size))/miSSt.size, cmap=plt.get_cmap('tab20'))

#         plt.scatter(miSStrs[fL2rs], miSSxrs[fL2rs], marker='+', alpha=1.0, color='black')
#         plt.scatter(miSStrs[oL2rs], miSSxrs[oL2rs], marker='+', alpha=1.0, color='blue')
        
#         plt.scatter(miSStjs[fL2js], miSSxjs[fL2js], marker='.', alpha=1.0, color='black')
#         plt.scatter(miSStjs[oL2js], miSSxjs[oL2js], marker='.', alpha=1.0, color='red')
        
#     return dSS1
# dSS = test(eDF, eDFc)
# #print(dSS)
        
# def shuffle_DF(DF):
#     D = {}
#     for c, L in [(c, list(DF[c])) for c in DF.columns]:
#         random.shuffle(L)
#         D[c] = L
#     return pd.DataFrame(D, DF.index)

# def join_split_DF(DF1, DF2):
#     c1, c2, DF1m, DF2m = DF1.shape[0], DF2.shape[0], DF1, DF2
#     if c1 < c2:
#         m, b = c2 // c1, c2 % c1
#         DF1m = pd.concat([DF1]*m + [DF1.iloc[range(b)]])  
#     elif c2 < c1:
#         m, b = c1 // c2, c1 % c2
#         DF2m = pd.concat([DF2]*m + [DF2.iloc[range(b)]])
#     jDF = pd.concat(DF1m, DF2m)    
#     rSS = pd.Series([random.choice([True, False]) for i in range(jDF.shape[0])], index=jDF.index)
#     jsDF1, jsDF2 = jDF.loc[rSS], jDF.loc[~rSS]
#     jsDF1.index, jsDF2.index = range(jsDF1.shape[0]), range(jsDF2.shape[0])
#     return jsDF1, jsDF2

# def get_outlier_filtered_mean_and_stdev(SS, hnorm=False):
#     if hnorm: SS = pd.Series(list(np.abs(SS)) + list(-np.abs(SS)))
#     q1, q2, q3 = SS.quantile([.25, .5, .75])
#     SS2 = SS[(q2-(q3-q1)*1.5 <= SS) & (SS < q2+(q3-q1)*1.5)]
#     return np.mean(SS2), np.std(SS2)

# def find_P_of_nontrivial_MI(miOt, miOc, miOx, miOtr, miOcr, miOxr):
    
#     icL, miSSt, miSSc, miSSx = None, miOt, miOc, miOx 
#     miSStr, miSScr, miSSxr = miOtr, miOcr, miOxr 
#     if type(miOt) is pd.DataFrame:
#         icL = [(i, c) for c in miOt for i in miOt.index if not pd.isna(miOt[c][i])]
#         miSSt = pd.Series([v for c in miOt for v in miOt[c] if not pd.isna(v)], index=icL)
#         miSSc = pd.Series([v for c in miOc for v in miOc[c] if not pd.isna(v)], index=icL)
#         miSSx = pd.Series([v for c in miOx for v in miOx[c] if not pd.isna(v)], index=icL)
#         miSStr = pd.Series([v for c in miOtr for v in miOtr[c] if not pd.isna(v)], index=icL)
#         miSScr = pd.Series([v for c in miOcr for v in miOcr[c] if not pd.isna(v)], index=icL)
#         miSSxr = pd.Series([v for c in miOxr for v in miOxr[c] if not pd.isna(v)], index=icL)
                    
#     mn_tr, std_tr = get_outlier_filtered_mean_and_stdev(miSStr)
#     mn_cr, std_cr = get_outlier_filtered_mean_and_stdev(miSScr)
#     dSStr, dSScr = miSStr-mn_tr, miSScr-mn_cr
#     miSStcr = np.sqrt(dSStr**2 + dSScr**2)
#     mn_tcr, std_tcr = get_outlier_filtered_mean_and_stdev(miSStcr, hnorm=True) 
#     dSSt, dSSc = miSSt-mn_tr, miSSc-mn_cr
#     miSStc = np.sqrt(dSSt**2 + dSSc**2)
#     tSStc = np.abs(miSStc-mn_tcr)/std_tcr
    
#     mn_xr, std_xr = get_outlier_filtered_mean_and_stdev(miSSxr)
#     tSSx = np.abs(miSSx-mn_xr)/std_tcr
    
    
#     tSS = pd.Series([min([tSStc[i], tSSx[i]]) for i in tSStc.index], index=tSStc.index)
#     tSS2 = pd.Series([max([tSStc[i], tSSx[i]]) for i in tSStc.index], index=tSStc.index)
    
#     print(len(tSStc[tSStc<3]), len(tSSx[tSSx<3]), len(tSS[tSS<3]), len(tSS2[tSS2<3]))
    
#     #print(pd.DataFrame({'tSStc': tSStc, 'tSSx': tSSx, #'tSS': tSS
#     #                   }, index=tSStc.index))
 
#     pO = pd.Series(1.0-stats.t.sf(tSS, len(tSS)-1), index=tSS.index)
#     if type(miOt) is pd.DataFrame:
#         pD = {c: [np.NaN]*miOt.shape[1] for c in miOt.columns}
#         for i, c in icL: pD[c][miOt.index.get_loc(i)] = pO[(i, c)]
#         pO = pd.DataFrame(pD, miOt.index)
#     return pO        

# def find_P_of_nontrivial_MI_diff(miOt, miOc, miOx, miOtr, miOcr, miOxr, 
#                                  miOtjs, miOcjs, miOxjs, pO): 
    
#     icL, miSSt, miSSc, miSSx = None, miOt, miOc, miOx
#     miSStr, miSScr, miSSxr = miOtr, miOcr, miOxr
#     miSStjs, miSScjs, miSSxjs, pSS = miOtjs, miOcjs, miOxjs, pO 
#     if type(miOt) is pd.DataFrame:
#         icL = [(i, c) for c in miOt for i in miOt.index if not pd.isna(miOt[c][i])]
#         miSSt = pd.Series([v for c in miOt for v in miOt[c] if not pd.isna(v)], index=icL)
#         miSSc = pd.Series([v for c in miOc for v in miOc[c] if not pd.isna(v)], index=icL)
#         miSSx = pd.Series([v for c in miOx for v in miOx[c] if not pd.isna(v)], index=icL)

#         miSStr = pd.Series([v for c in miOtr for v in miOtr[c] if not pd.isna(v)], index=icL)
#         miSScr = pd.Series([v for c in miOcr for v in miOcr[c] if not pd.isna(v)], index=icL)
#         miSSxr = pd.Series([v for c in miOxr for v in miOxr[c] if not pd.isna(v)], index=icL)

#         miSStjs = pd.Series([v for c in miOtjs for v in miOtjs[c] if not pd.isna(v)], index=icL)
#         miSScjs = pd.Series([v for c in miOcjs for v in miOcjs[c] if not pd.isna(v)], index=icL)
#         miSSxjs = pd.Series([v for c in miOxjs for v in miOxjs[c] if not pd.isna(v)], index=icL)
#         pSS = pd.Series([v for c in pO for v in pO[c] if not pd.isna(v)], index=icL)
            
#     dSStc = (miSStjs-miSScjs)[pSS>0.997]
#     mn_tc, std_tc = get_outlier_filtered_mean_and_stdev(dSStc, hnorm=True)
#     tSStc = np.abs((miSSt-miSSc)/std_tc)
    
#     dSSx = (miSStjs-miSSxjs)[pSS>0.997]
#     mn_x, std_x = get_outlier_filtered_mean_and_stdev(dSSx)
#     tSSx = np.abs(((miSSt-miSSx)-mn_x)/std_x)

#     tSS = pd.Series([min([tSStc[i], tSSx[i]]) for i in tSStc.index], index=tSStc.index)
    
#     print('all')
#     print(pd.DataFrame({'tSStc': tSStc[pSS>0.997], 
#                         'tSSx': tSSx[pSS>0.997], 
#                        },
#                        index=tSStc[pSS>0.997].index))
#     print()
#     print('tc:')
#     print(pd.DataFrame({'tSStc': tSStc[(pSS>=0.997) & (tSStc>=3)], 
#                         'tSSx': tSSx[(pSS>=0.997) & (tSStc>=3)],
#                        },
#                        index=tSStc[(pSS>=0.997) & (tSStc>=3)].index))
#     print()
#     print('x:')
#     print(pd.DataFrame({'tSStc': tSStc[(pSS>=0.997) & (tSSx>=3)], 
#                         'tSSx': tSSx[(pSS>=0.997) & (tSSx>=3)], 
#                        },
#                        index=tSSx[(pSS>=0.997) & (tSSx>=3)].index))
    
#     pO = pd.Series(1.0-stats.t.sf(tSS, len(tSS)-1), index=tSS.index)
#     if type(miOt) is pd.DataFrame:
#         pD = {c: [np.NaN]*miOt.shape[1] for c in miOt.columns}
#         for i, c in icL: pD[c][miOt.index.get_loc(i)] = pO[(i, c)]
#         pO = pd.DataFrame(pD, miOt.index)
#     return pO
        
# def get_bkg_colormap():
#     cMap = cc.cm['bkr']
#     sdD, sdD2 = cMap.__dict__['_segmentdata'], {}
#     r1, r2 = sdD['red'][:128], sdD['red'][128:]
#     b1, b2 = sdD['blue'][:128], sdD['blue'][128:]
#     g1, g2 = sdD['green'][:128], sdD['green'][128:]    
#     sdD2['red'] = r1 + [(r2[i][0], *r1[127-i][1:]) for i in range(128)]
#     sdD2['blue'] = b1 + [(b2[i][0], *g1[127-i][1:]) for i in range(128)]
#     sdD2['green'] = g1 + [(g2[i][0], *b1[127-i][1:]) for i in range(128)]
#     return cc.LinearSegmentedColormap('bkg',  sdD2)

# def get_yky_colormap():
#     cMap, e = cc.cm['bkr'], 4
#     sdD, sdD2 = cMap.__dict__['_segmentdata'], {}    
#     r1, r2 = sdD['red'][:128], sdD['red'][128:]
#     g1, g2 = sdD['green'][:128], sdD['green'][128:]        
#     b1, b2 = sdD['blue'][:128], sdD['blue'][128:]    
#     sdD2['red'] = [(r1[i][0], r2[127-i][1]**e, r2[127-i][2]**e) for i in range(128)] \
#             + [(r2[i][0], r2[i][1]**e, r2[i][2]**e) for i in range(128)]
#     sdD2['green'] = [(r1[i][0], r2[127-i][1]**e, r2[127-i][2]**e) for i in range(128)] \
#             + [(r2[i][0], r2[i][1]**e, r2[i][2]**e) for i in range(128)]
#     sdD2['blue'] = [(b1[i][0], b2[127-i][1]**e, b2[127-i][2]**e) for i in range(128)] \
#             + [(b2[i][0], b2[i][1]**e, b2[i][2]**e) for i in range(128)]
#     return cc.LinearSegmentedColormap('yky',  sdD2)

# def _get_0norm_filtered_SS(SS):
#     sSS = SS.sort_values()
#     min_zNiL, max_dNiL, last_vz = [], [], None
#     for i in range(10, sSS.size):
#         mSS = pd.concat([sSS[:i], -sSS[:i]])
#         z, p = stats.kurtosistest(mSS)
#         std, v, z = np.std(mSS), sSS[i], abs(z)
#         vz = v/std
        
#         s = '%d. %.6f (p=%.6f)  ->  v=%.6f, std=%.6f : %.6f' % (i, z, p, v, std, vz)

#         if z and ((not min_zNiL) or z<=min_zNiL[-1][0]):
#             min_zNiL.append((z, i))
#             s += ' !'  

#         if last_vz and ((not max_dNiL) or vz-last_vz>=max_dNiL[-1][0]):
#             max_dNiL.append((vz-last_vz, i))
#             s += ' *'
            
#         if z == 0: s += ' X'
#         print(s)
        
#         last_vz = vz
        
#     min_zi = min_zNiL[-1][1]+1
#     i = ([min_zi]+[dNi[1] for dNi in max_dNiL if dNi[1] <= min_zi])[-1] 
#     print(min_zi, i)
#     return sSS[:i]

#def test(tDF, cDF):
#    tDFr, cDFr = _synthesize_by_field_DF(tDF), _synthesize_by_field_DF(cDF)
#    miSStr, miSScr = get_3v_MIs_for_DF(tDFr), get_3v_MIs_for_DF(cDFr)
#    dSS_mi0n = _get_0norm_filtered_SS(np.sqrt(miSStr**2+miSScr**2))
#    return dSS_mi0n
#test(eDF, eDFc)
    
#     tDFr2, cDFr2 = _synthesize_by_field_DF(tDF), _synthesize_by_field_DF(cDF)
#     miSStr2, miSScr2 = get_3v_MIs_for_DF(tDFr2), get_3v_MIs_for_DF(cDFr2) 
#     miSSxr2 = get_3v_MIs_for_DF(tDFr2, cDFr2) 
#     if True:
#         plt.figure(1, figsize=(12, 12))
#         plt.subplot(2, 2, 1)
#         plt.scatter(miSStr, miSStr2, marker='.',
#                     c=pd.Series(range(miSStr.size))/miSStr.size, cmap=plt.get_cmap('tab20'))
#         plt.subplot(2, 2, 2)
#         plt.scatter(miSScr, miSScr2, marker='.',
#                     c=pd.Series(range(miSScr.size))/miSScr.size, cmap=plt.get_cmap('tab20'))
#         plt.figure(1, figsize=(12, 12))
#         plt.subplot(2, 2, 3)
#         plt.scatter(miSSxr, miSSxr2, marker='.',
#                     c=pd.Series(range(miSSxr.size))/miSSxr.size, cmap=plt.get_cmap('tab20'))

#     if False:
#         plt.figure(1, figsize=(12, 24))
#         plt.subplot(4, 2, 1)
#         plt.scatter(tSStc_eq, tSSx_eq, marker='.', color='white')
#         plt.scatter(tSStc_no, tSSx_no, marker='.', color='green')
#         #plt.axis('equal')

#         plt.subplot(4, 2, 3)
#         plt.scatter(miSStr, miSScr, marker='.', color='yellow')

#         plt.scatter([miSStr.mean()], [miSScr.mean()], color='pink')
        
#         mn_js1, std_js1 = _get_outlier_filtered_mean_and_stdev(miSSjs1)
#         mn_js2, std_js2 = _get_outlier_filtered_mean_and_stdev(miSSjs2)
#         b, m = 0, mn_js2/mn_js1
#         #b, m = polyfit(miSSjs1, miSSjs2, 1)
#         plt.plot(miSSjs1, b+m*miSSjs1, '-', color='orange', alpha=0.5)

#         plt.scatter(miSSjs1, miSSjs2, marker='.', color='orange')
#         plt.axis('equal')

#         plt.subplot(4, 2, 5)
#         #plt.scatter(miSStr, miSScr, marker='.', color='yellow', alpha=0.25)

#         plt.scatter([miSStr.mean()], [miSScr.mean()], color='pink')
#         mn_js1, std_js1 = _get_outlier_filtered_mean_and_stdev(miSSjs1)
#         mn_js2, std_js2 = _get_outlier_filtered_mean_and_stdev(miSSjs2)
#         b, m = 0, mn_js2/mn_js1
#         #b, m = polyfit(miSSjs1, miSSjs2, 1)
#         plt.plot(miSSjs1, b+m*miSSjs1, '-', color='orange', alpha=0.5)
#         plt.scatter(miSSjs1, miSSjs2, marker='.', color='orange', alpha=0.25)
        
#         mn_t, std_t = _get_outlier_filtered_mean_and_stdev(miSSt)
#         mn_c, std_c = _get_outlier_filtered_mean_and_stdev(miSSc)
#         b2, m2 = 0, mn_c/mn_t
#         #b2, m2 = polyfit(miSSt, miSSc, 1)
#         plt.plot(miSSjs1, b2+m2*miSSjs1, '-', color='red', alpha=0.5)
#         plt.scatter(miSSt, miSSc, marker='.', color='red')
#         plt.scatter(miSSt[fSS_eq], miSSc[fSS_eq], marker='.', color='gray')
#         #plt.scatter(miSSt[fSS_no], miSSc[fSS_no], marker='.', color='black')
#         plt.axis('equal')

#         plt.subplot(4, 2, 7)
#         #plt.scatter(miSStr, miSScr, marker='.', color='yellow', alpha=0.25)
#         plt.scatter(miSSjs1, miSSjs2, marker='.', color='orange', alpha=0.25)
#         plt.scatter(miSSt, miSSc, marker='.', color='red')
#         plt.scatter(miSSt[fSS_eq2], miSSc[fSS_eq2], marker='.', color='gray')
#         #plt.scatter(miSSt[fSS_no2], miSSc[fSS_no2], marker='.', color='black')
#         plt.axis('equal')
        
#         plt.subplot(4, 2, 2)
#         plt.scatter(tSStc_no, tSSx_no, marker='.', color='white')
#         plt.scatter(tSStc_eq, tSSx_eq, marker='.', color='green')
#         #plt.axis('equal')

#         plt.subplot(4, 2, 4)
#         plt.scatter(miSStr, miSSxr, marker='.', color='yellow')
#         plt.scatter([miSStr.mean()], [miSSxr.mean()], color='pink')
        
#         mn_js1, std_js1 = _get_outlier_filtered_mean_and_stdev(miSSjs1)
#         mn_jsX, std_jsX = _get_outlier_filtered_mean_and_stdev(miSSjsX)
#         b, m = 0, mn_jsX/mn_js1
#         #b, m = polyfit(miSSjs1, miSSjsX, 1)
#         plt.plot(miSSjs1, b+m*miSSjs1, '-', color='orange', alpha=0.5)
#         plt.scatter(miSSjs1, miSSjsX, marker='.', color='orange')
#         #plt.axis('equal')

#         plt.subplot(4, 2, 6)
#         #plt.scatter(miSStr, miSSxr, marker='.', color='yellow', alpha=0.25)
#         plt.scatter(miSSjs1, miSSjsX, marker='.', color='orange', alpha=0.25)
#         plt.scatter(miSSt, miSSx, marker='.', color='red')
#         plt.scatter(miSSt[fSS_eq], miSSx[fSS_eq], marker='.', color='gray')
#         #plt.scatter(miSSt[fSS_no], miSSx[fSS_no], marker='.', color='black')
#         #plt.axis('equal')

#         plt.subplot(4, 2, 8)
#         #plt.scatter(miSStr, miSSxr, marker='.', color='yellow', alpha=0.25)
#         plt.scatter([miSStr.mean()], [miSSxr.mean()], color='pink')
        
#         mn_js1, std_js1 = _get_outlier_filtered_mean_and_stdev(miSSjs1)
#         mn_jsX, std_jsX = _get_outlier_filtered_mean_and_stdev(miSSjsX)
#         b, m = 0, mn_jsX/mn_js1
#         #b, m = polyfit(miSSjs1, miSSjsX, 1)    
#         plt.plot(miSSjs1, b+m*miSSjs1, '-', color='orange', alpha=0.5)
#         plt.scatter(miSSjs1, miSSjsX, marker='.', color='orange', alpha=0.25)

#         mn_t, std_t = _get_outlier_filtered_mean_and_stdev(miSSt)
#         mn_x, std_x = _get_outlier_filtered_mean_and_stdev(miSSx)
#         b2, m2 = 0, mn_x/mn_t
#         #b2, m2 = polyfit(miSSt, miSSx, 1)
#         plt.plot(miSSjs1, b2+m2*miSSjs1, '-', color='red', alpha=0.5)
#         plt.scatter(miSSt, miSSx, marker='.', color='red')
#         plt.scatter(miSSt[fSS_eq2], miSSx[fSS_eq2], marker='.', color='gray')
#         #plt.scatter(miSSt[fSS_no2], miSSx[fSS_no2], marker='.', color='black')
#         #plt.axis('equal')
    
#     plt.figure(1, figsize=(10, 20))
#     plt.subplot(2,1,1)
#     miSSd = (miSSt-miSSc)#/np.sqrt(miSSt**2+miSSc**2)
#     plt.scatter(miSSt, miSSd, marker='.', color='orange')
#     plt.scatter(miSSt[(pSS<0.997)], miSSd[(pSS<0.997)], marker='.', color='black')
#     plt.scatter(miSSt[(tSStc*tSSx>=9)], miSSd[(tSStc*tSSx>=9)], marker='+', color='red')

#     plt.subplot(2,1,2)
#     miSSd2 = miSSx
#     plt.scatter(miSSt[(pSS>=0.997)], miSSd2[(pSS>=0.997)], marker='.', color='orange')
#     plt.scatter(miSSt[(pSS<0.997)], miSSd2[(pSS<0.997)], marker='.', color='black')
#     plt.scatter(miSSt[(tSStc*tSSx>=9)], miSSd2[(tSStc*tSSx>=9)], marker='+', color='red')

    
#     rSS = pd.Series([random.random() for i in range(miSSx.size)], index=miSSx.index)
#     plt.scatter((miSSx-miSSxjs), rSS, marker='.', color='orange')
#     plt.scatter((miSSx-miSSxjs)[(pSS<0.997)], rSS[(pSS<0.997)], marker='.', color='black')
#     plt.scatter((miSSx-miSSxjs)[(tSStc*tSSx>=9)], rSS[(tSStc*tSSx>=9)], marker='+', color='red')
    
    #plt.scatter(np.abs(miSSt-miSSc), np.abs(miSSx), marker='.', color='orange')
    #plt.scatter(np.abs(miSSt-miSSc)[(pSS<0.997)], np.abs(miSSx)[(pSS<0.997)], 
    #            marker='.', color='black')
    #plt.scatter(np.abs(miSSt-miSSc)[(tSStc*tSSx>=9)], np.abs(miSSx)[(tSStc*tSSx>=9)], 
    #            marker='+', color='red')

    #plt.figure(1, figsize=(10, 30))
    #plt.subplot(3, 1, 1)
    #plt.scatter(miSStr, miSScr, marker=".", color='deepskyblue', alpha=0.1)
    #plt.scatter(miSStjs, miSScjs, marker='.', color='limegreen', alpha=0.3)
    #plt.scatter(miSSt, miSSc, marker='.', color='orange', alpha=0.3)

    #plt.scatter(miSStjs, miSScjs, marker='.', color='yellow', alpha=0.5)
    #plt.scatter(miSSt[tSS<3], miSSc[tSS<3], marker='.', color='deepskyblue', alpha=0.5)
    #plt.scatter(miSSt[tSS>=3], miSSc[tSS>=3], marker='.', color='blue', alpha=1)
    #plt.scatter(miSSt[(tSSx>=3) & (tSStc<3)], miSSc[(tSSx>=3) & (tSStc<3)], 
    #            marker='.', color='red', alpha=1)
    #plt.scatter(miSSt[(tSStc>=3) & (tSSx<3)], miSSc[(tSStc>=3) & (tSSx<3)], 
    #            marker='.', color='limegreen', alpha=1)
    #plt.scatter(miSSt[(pSS<0.997)], miSSc[(pSS<0.997)], 
    #            marker='.', color='black', alpha=0.5)
    #plt.scatter(miSSt[(tSStc*tSSx>=9)], miSSc[(tSStc*tSSx>=9)], 
    #            marker='+', color='red', alpha=1)

    #plt.subplot(3, 1, 2)
    #plt.scatter(miSStr, miSSxr, marker='.', color='deepskyblue', alpha=0.3)
    #plt.scatter(miSStjs, miSSxjs, marker='.', color='limegreen', alpha=0.3)
    #plt.scatter(miSSt, miSSx, marker='.', color='orange', alpha=0.3)
    #plt.scatter(miSStjs, miSSxjs, marker='.', color='yellow', alpha=0.5)
    #plt.scatter(miSSt[tSS<3], miSSx[tSS<3], marker='.', color='deepskyblue', alpha=0.5)
    #plt.scatter(miSSt[tSS>=3], miSSx[tSS>=3], marker='.', color='blue', alpha=1)
    #plt.scatter(miSSt[(tSSx>=3) & (tSStc<3)], miSSx[(tSSx>=3) & (tSStc<3)], 
    #            marker='.', color='red', alpha=1)
    #plt.scatter(miSSt[(tSStc>=3) & (tSSx<3)], miSSx[(tSStc>=3) & (tSSx<3)], 
    #            marker='.', color='limegreen', alpha=1)
    #plt.scatter(miSSt[(pSS<0.997)], miSSx[(pSS<0.997)], 
    #            marker='.', color='black', alpha=0.5)
    #plt.scatter(miSSt[(tSStc*tSSx>=9)], miSSx[(tSStc*tSSx>=9)], 
    #            marker='+', color='red', alpha=1)

    #plt.subplot(3, 1, 3)
    #plt.scatter(tSStc[tSS<3], tSSx[tSS<3], marker='.', color='deepskyblue', alpha=0.5)
    #plt.scatter(tSStc[tSS>=3], tSSx[tSS>=3], marker='.', color='blue', alpha=1)
    #plt.scatter(tSStc[(tSSx>=3) & (tSStc<3)], tSSx[(tSSx>=3) & (tSStc<3)], 
    #           marker='.', color='red', alpha=1)
    #plt.scatter(tSStc[(tSStc>=3) & (tSSx<3)], tSSx[(tSStc>=3) & (tSSx<3)], 
    #           marker='.', color='limegreen', alpha=1)
    #plt.scatter(tSStc[(tSStc*tSSx>=9)], tSSx[(tSStc*tSSx>=9)], 
    #           marker='+', color='black', alpha=1)
    #plt.scatter(tSStc[(pSS<0.997)], tSSx[(pSS<0.997)], 
    #           marker='.', color='gray', alpha=0.5)
    
# def get_E_for_SS(SS, cSS=None, tflag=None):    
#     p_counter = collect.Counter(SS)
#     if len(p_counter) <= 1:
#         raise AssertionError('Target Series must have at least 2 states')
#     e = None
#     if cSS is None:
#         e = stats.entropy(list(p_counter.values()), base=2)
#     elif cSS is not None:
#         q_counter = collect.Counter(cSS) 
#         if len(q_counter) <= 1:
#             raise AssertionError('Reference Series must have at least 2 states')
#         pk, qk = [], []
#         for k in pd.Series(sorted(list(set( p_counter.keys()) | set(q_counter.keys()) ))):
#             if k in p_counter and k in q_counter:
#                 pk.append(p_counter[k])
#                 qk.append(q_counter[k])
#             elif k not in p_counter and k in q_counter:
#                 pk.append(0)
#                 qk.append(q_counter[k])
#             elif k in p_counter and k not in q_counter:    
#                 pk.append(0)
#                 qk.append(p_counter[k])
#                 #raise AssertionError(
#                 #        'Kullback-Leibler requirement: ' + 
#                 #        'Reference Membership must contain Target Membership'
#                 #    )
#         e = stats.entropy(pk, base=2) + stats.entropy(pk, qk=qk, base=2)
#     return e
    
# def get_E_for_SS(SS, cSS=None, refEntropy=None, tflag=None):
#     p_counter = collect.Counter(SS)
#     if len(p_counter) <= 1:
#         raise AssertionError('Target Series must have at least 2 states')
#     e = None
#     if cSS is not None:
#         q_counter = collect.Counter(cSS) 
#         if len(q_counter) <= 1:
#             raise AssertionError('Reference Series must have at least 2 states')
#         pk, qk = [], []
#         for k in pd.Series(sorted(list(set( p_counter.keys()) | set(q_counter.keys()) ))):
#             if k in p_counter and k in q_counter:
#                 pk.append(p_counter[k])
#                 qk.append(q_counter[k])
#             elif k not in p_counter and k in q_counter:
#                 pk.append(0)
#                 qk.append(q_counter[k])
#             elif k in p_counter and k not in q_counter:    
#                 raise AssertionError(
#                         'Kullback-Leibler requirement: ' + 
#                         'Reference Membership must contain Target Membership'
#                     )
#         e = stats.entropy(pk, base=2) + stats.entropy(pk, qk=qk, base=2)
#         if tflag and tflag == 'alt':
#             e = stats.entropy(pk, qk=qk, base=2)
#     elif refEntropy is not None:
#         e = stats.entropy(list(p_counter.values()), base=2) - refEntropy        
#     elif cSS is None and refEntropy is None:
#         e = stats.entropy(list(p_counter.values()), base=2)
#     return e

# b, m = polyfit(miSSt, miSSc, 1)

# mdn, iqr = dSS1.median(), abs(dSS1.quantile(0.75)-dSS1.quantile(0.25))
# miSSt_iqr = miSSt[(mdn - (iqr*1.5) <= dSS1) & (dSS1 < mdn + (iqr*1.5))]
# miSSc_iqr = miSSc[(mdn - (iqr*1.5) <= dSS1) & (dSS1 < mdn + (iqr*1.5))]
# b2, m2 = polyfit(miSSt_iqr, miSSc_iqr, 1)

# plt.plot(miSSt, miSSt, '-', color='yellow', alpha=0.5)
# plt.plot(miSSt, b+m*miSSt, '-', color='orange', alpha=0.5)
# plt.plot(miSSt, b2+m2*miSSt, '-', color='red', alpha=0.5)
    
#print(list(miSSt))
#print(min(miSSt), max(miSSt))
#print(list(miSSc))
#print(min(miSSc), max(miSSc))

#b, m = polyfit(miSSt, miSSc, 1)
#m2 = miSSt.dot(miSSc)/miSSt.dot(miSSt)
#m3 = miSSc.mean()/miSSt.mean()
#m4 = miSSc.median()/miSSt.median()
#print(m, m2, m3)
#plt.plot(miSSt, miSSt, '-')
#plt.plot(miSSt, b + m * miSSt, '-')
#plt.plot(miSSt, m3*miSSt, '-')
#plt.plot(miSSt, m4*miSSt, '-')
#plt.scatter(miSSt, miSSc, marker='.', c=catSS1b, alpha=0.5)
#plt.plot([miSSt.mean()], [miSSc.mean()], 'o', c='black')
#print(b, m)
#x = miSSt.mean()
#print(x, b+m*x, m3*x)

#tDF[tDF['t-c']]
    
        
# LETTER_COLOR_DICT = {
#         'A':'coral', 'B': 'gold', 'C': 'springgreen', 'D': 'dodgerblue', 
#         'E': 'lavender', 'F': 'pink', 'G': 'chocolate', 'H': 'orange', 
#         'I': 'teal', 'J': 'cadetblue', 'K': 'blueviolet', 'L': 'purple',
#         'M': 'rosybrown', 'N': 'goldenrod', 'O': 'lawngreen', 'P': 'aqua', 
#         'Q': 'slateblue', 'R': 'orchid', 'S': 'palevioletred', 'T': 'peachpuff', 
#         'U': 'r', 'V': 'y', 'W': 'g', 'X': 'c', 'Y': 'b', 'Z': 'm'
#     }
        
# def entropy2rank(pdO):
#     pdO2 = None
#     if type(pdO) is pd.Series:
#         pdO2 = pd.Series([v for v in stats.rankdata(pdO)], index=pdO.index)
#     elif type(pdO) is pd.DataFrame:
#         rankL = stats.rankdata([v for c in pdO.columns for v in pdO[c] if not pd.isna(v)])
#         D, i = {}, 0
#         for c in pdO.columns:
#             L = []
#             for r in pdO.index:
#                 if pd.isna(pdO[c][r]): L.append(None)
#                 else: 
#                     L.append(rankL[i])
#                     i += 1
#             D[c] = L
#         pdO2 = pd.DataFrame(D, columns=pdO.columns, index=pdO.index)
#     return pdO2

#eOB_ref = get_unique_values_for_DF(eDFm)

# legendValue = get_jE_for_DF(eDF_target)
# legendValue2 = get_jE_for_DF(eDF_target, eOB_ref)
# print("E=%f, E_kl=%f" % (legendValue, legendValue2))
# print('---')

# pairDFa = get_2v_MIs_for_DF(eDF_target)
# pair2DFa = get_2v_MIs_for_DF(eDF_target, cDF=eOB_ref)
# print('MI2:')
# print(pairDFa.values.tolist())
# print('---')
# print('MI2_kl:')
# print(pair2DFa.values.tolist())
# print('---')

# tripletsSSa = get_3v_MIs_for_DF(eDF_target)
# triplets2SSa = get_3v_MIs_for_DF(eDF_target, cDF=eOB_ref)
# print('MI3:')
# print(tripletsSSa.tolist())
# print('---')
# print('MI3_kl:')
# print(triplets2SSa.tolist())
# print('---')

# plt.axvline(0, color='black', linestyle='dashed', alpha=0.5)
# plt.hist(tripletsSSa, label='MI3')
# plt.hist(triplets2SSa, label='MI3_kl')
# plt.legend(facecolor='black')
# matplotlib.rcParams.update({'text.color': 'white'})
#plt.plot(tripletsSSa, triplets2SSa, 'bo', tripletsSSa, a, 'gs') #, a, triplets2SSa, 'r^')
#plt.axis('equal')

#render_entropy_graph(eDFm, cDF=eDF2m, outfile='test.png')
    
    

#def get_E_for_SS2(SS, cSS=None, keySS=None, refEntropy=None):
#    p_counter = collect.Counter(SS)
#    e = None
#    if cSS is not None:
#        q_counter = collect.Counter(cSS)
#        pk, qk = [], []
#        if keySS is None: keySS = pd.Series(sorted(list(set( p_counter.keys()) | set(q_counter.keys()) ))) 
#        for k in keySS:
#            pk.append(((p_counter[k] if k in p_counter else 0) + 0.00001))
#            qk.append(((q_counter[k] if k in q_counter else 0) + 0.00001))
#        e = -stats.entropy(pk, qk=qk, base=2)    
#    elif refEntropy is not None:
#         e = stats.entropy(list(p_counter.values()), base=2) - refEntropy
#     elif cSS is None and refEntropy is None:
#         e = stats.entropy(list(p_counter.values()), base=2)
#     return e

#def get_E_for_SS2(SS, cSS=None):
#    pk, qk = list(collect.Counter(SS).values()), list(collect.Counter(cSS).values()) if cSS is not None else None 
#    return 0.0 if SS.empty else stats.entropy(pk, qk=qk, base=2)

#def get_Es_for_DF2(DF, refDF = None):
#    return DF.apply(get_E_for_SS)

#q_counter = collect.Counter(cSS)
#qk = list(q_counter.values())
#pk.extend([0]*(len(qk)-len(pk)))
#e2 = stats.entropy(qk, base=2)
# ----------------
#e3 = stats.entropy(pk, qk=qk, base=2)
#if round(e3, 10) != round(abs(e1-e2), 10): 
#    print(round(e3, 10), '==?', round(abs(e1-e2), 10), (e1, '-',  e2, '=', e1-e2))
#    raise AssertionError('Kullback-Leibler != difference in entropy between series')
# ----------------

#if cDF is not None:
#    pairs2DFm = pairs2DF.fillna(0.0) + pairs2DF.T.fillna(0.0)
#    steppedSingletonsDF_color = sceDF.apply(lambda SS: SS.apply(lambda T: pairs2DFm[SS.name][T[0]]), axis=1)

#te = None
#if cDF is None:
#    te = get_E_for_SS(DF[target_field])    
#else:
#    total_E = get_E_for_SS(DF[target_field])
#    total_kl_divergence = get_E_for_SS(DF[target_field], cSS=cDF[target_field])
#legend_value1 = total_E - total_kl_divergence
#legend_value2 = total_E                                                 
#scale = get_scale(total_E, RADIUS_FOR_TOTAL_MUTUAL_INFORMATION)
#title, label = GRAPH_TITLE_2 % (target_field), GRAPH_DATASET_LABEL_2 % (target_field, total_E)        
#mutualinfoSS = get_2v_MIs_for_DF_and_target_field(DF, target_field)
#kldivergenceSS = get_2v_MIs_for_DF_and_target_field(DF, target_field, cDF=cDF)
#background_singletons1_SS = mutualinfoSS - kldivergenceSS
#background_singletons2_SS = mutualinfoSS
#scmiDF = get_stepped_cMI_data_for_DF_and_target_field(DF,
#            target_field, cDF=cDF)
#imiDF, cmiDF = scmiDF.applymap(lambda T: T[1]), scmiDF.applymap(lambda T: T[2])
#color_stepped_singletons_DF, size_stepped_singletons_DF = imiDF, cmiDF
#field_by_field_pairs_DF = get_3v_MIs_for_DF_and_target_field(DF,
#            target_field, cDF=cDF)
#triplets_SS = -get_4v_MIs_for_DF_and_target_field(DF,
#            target_field, cDF=cDF)

#def get_E_for_SS2(SS, cSS=None, keySS=None, refEntropy=None, smooth_exp=1.0):
#    p_counter = collect.Counter(SS)
#    e = None
#    if cSS is not None:
#        q_counter = collect.Counter(cSS)
#        pk, qk = [], []
#        if keySS is None: keySS = pd.Series(sorted(list(set( p_counter.keys()) | set(q_counter.keys()) ))) 
#        for k in keySS:
#            pk.append(((p_counter[k] if k in p_counter else 0) + 0.00001))
#            qk.append(((q_counter[k] if k in q_counter else 0) + 0.00001)**smooth_exp)
#        e = -stats.entropy(pk, qk=qk, base=2)    
#    elif refEntropy is not None:
#        e = stats.entropy(list(p_counter.values()), base=2) - refEntropy
#    elif cSS is None and refEntropy is None:
#        e = stats.entropy(list(p_counter.values()), base=2)
#    return e

#def get_jE_for_DF2(DF, cDF=None, smooth_exp=1.0):
#    SS = superkey_DF(DF)
#    cSS, keySS, refEntropy = None, None, None
#    if type(cDF) == pd.DataFrame:
#        kSSS1, kSSS2 = get_unique_values_for_DF(DF), get_unique_values_for_DF(cDF)
#        kSSS = pd.Series([pd.Series(sorted(set(kSSS1[c]) | set(kSSS2[c]))) for c in DF.columns], index=DF.columns)
#        keySS = cross_superkey_SS_of_SS(kSSS)
#        cSS = superkey_DF(cDF)
#    elif type(cDF) == pd.Series and not (False in [type(o) == pd.Series for o in cDF]):
#        p = func.reduce(operate.mul, [len(SS) for SS in cDF])
#        if p <= 100000:
#            cSS = cross_superkey_SS_of_SS(cDF)
#        else:
#            refEntropy = math.log2(p)
#    return get_E_for_SS2(SS, cSS=cSS, keySS=keySS, refEntropy=refEntropy, smooth_exp=smooth_exp)

#def get_Es_for_DF2(DF, cDF=None, smooth_exp=1.0):
#    cL, eL = DF.columns, None
#    if cDF is None: eL = [get_E_for_SS2(DF[c]) for c in cL]
#    else: eL = [get_E_for_SS2(DF[c], cSS=cDF[c], smooth_exp=smooth_exp) for c in cL]
#    return pd.Series(eL, index=cL)

#def get_2v_jEs_for_DF2(DF, cDF=None, smooth_exp=1.0):
#    cDje = {}
#    for i, c in enumerate(DF.columns):
#        cDje[c] = []
#        for j, c2 in enumerate(DF.columns):
#            if cDF is None:
#                cDje[c].append(get_jE_for_DF2(DF[[c, c2]]) if i > j else np.NaN)
#            else:
#                cDje[c].append(get_jE_for_DF2(DF[[c, c2]], cDF=cDF[[c, c2]], 
#                            smooth_exp=smooth_exp) if i > j else np.NaN)
#    return pd.DataFrame(cDje, columns=DF.columns, index=DF.columns)

#def get_2v_MIs_for_DF2(DF, cDF=None, smooth_exp=1.0):
#    eSS = get_Es_for_DF2(DF, cDF=cDF, smooth_exp=smooth_exp)
#    jeDF = get_2v_jEs_for_DF2(DF, cDF=cDF, smooth_exp=smooth_exp)    
#    return pd.DataFrame({c: [eSS[c]+eSS[c2]-jeDF[c][c2] for c2 in DF.columns] for c in DF.columns}, index = DF.columns)

#def get_3v_jEs_for_DF2(DF, cDF=None, smooth_exp=1.0):
#    c1c2c3L, je3L = [], []
#    for i in range(DF.shape[1]):
#        for j in range(i+1, DF.shape[1]):
#            for k in range(j+1, DF.shape[1]):
#                c1, c2, c3, je3 = DF.columns[i], DF.columns[j], DF.columns[k], None
#                if cDF is None:
#                    je3 = get_jE_for_DF2( DF[[c1,c2,c3]] )
#                else:
#                    je3 = get_jE_for_DF2(DF[[c1,c2,c3]], cDF=cDF[[c1,c2,c3]], smooth_exp=smooth_exp)
#                c1c2c3L.append((c1,c2,c3))
#                je3L.append(je3)
#    return pd.Series(je3L, index=c1c2c3L)

#def get_3v_MIs_for_DF2(DF, cDF=None, smooth_exp=1.0):
#    eSS = get_Es_for_DF2(DF, cDF=cDF, smooth_exp=smooth_exp)
#    jeDF = get_2v_jEs_for_DF2(DF, cDF=cDF, smooth_exp=smooth_exp)
#    tjeSS = get_3v_jEs_for_DF2(DF, cDF=cDF,smooth_exp=smooth_exp)    
#    tmiL, cxcyczL = [], []
#    for i in range(DF.shape[1]):
#        for j in range(i+1, DF.shape[1]):
#            for k in range(j+1, DF.shape[1]):
#                cX, cY ,cZ = DF.columns[i], DF.columns[j], DF.columns[k]
#                eX, eY, eZ = eSS[cX], eSS[cY], eSS[cZ]
#                jeXY, jeXZ, jeYZ = jeDF[cY][cX], jeDF[cZ][cX], jeDF[cZ][cY]  
#                tje = tjeSS[(cX,cY,cZ)]
#                tmi = tje+eX+eY+eZ-jeXZ-jeXY-jeYZ
#                tmiL.append(tmi)
#                cxcyczL.append((cX,cY,cZ))
#    return pd.Series(tmiL, index=cxcyczL)